# Taxi Demand Prediction - Porto city - Portugal

## Business/Real World Problem
For a given Taxi stand in Porto, our goal is to <b>predict the number of pickups in that given stand</b>. Some location require more taxis at a particular time than other locations owing to the presence schools, hospitals, offices etc. The prediction result can be transferred to the taxi drivers via Smartphone app, and they can subsequently move to the stands where predicted pickups are high.

## Objectives & Constraints
<b>Objectives:</b> Our objective is to predict the number of pickups as accurately as possible for each taxi stand in a 30-min interval. We have 63 taxi station in Porto city.<br><br>
<b>Constraints:</b>
* <b>Latency:</b> Given a location and current time of a taxi driver, as a taxi driver, he/she excepts to get the predicted pickups in his/her stand and the adjoining stands in few seconds. Hence, there is a medium latency requirement.<br><br>

* <b>Interpretability:</b> As long as taxi driver gets good prediction result, he/she is not be much interested in the interpretability of the result. He/she is not much interested in why he/she is getting this result. Hence, there is a no interpretability required.<br><br>

* <b>Relative Errors:</b> Symmetric Mean Absolute Percentage Error will be the relative error we will consider. Let say the predicted pickups for a particular station are 100, but actual pickups are 102, the percentage error will be 2% and Absolute error is 2. The taxi driver will be more interested in the percentage error than the absolute error. Let say in some stand the predicted pickups are 250, and if taxi driver knows that the relative error is 10% then he/she will consider the predicted result to be in the range of 225 to 275, which is considerable.<br><br>

<b>Our goal is to reduce the percentage error as low as possible.</b> 

# 1. Data Information
Porto taxi dataset
https://www.kaggle.com/c/pkdd-15-predict-taxi-service-trajectory-i/data
We have used 01 Jun 2013 and 30 Jun 2014 data.


<table>
    <tr>
        <th>File-Name</th>
        <th>File-Size</th>
        <th>Number of Records (before)</th>
        <th>Number of Records (after)</th>
        <th>Number of Features</th>
        </tr>
    <tr>
        <td> data_prcess.csv</td>
        <td> 116MB</td>
        <td> 1.710.670</td>
        <td> 1.706.572</td>
        <td> 8 </td>
    </tr>
    

In [ ]:
import dask.dataframe as dd
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np
import folium
import gpxpy.geo
from datetime import datetime
import time
import seaborn as sns
import os
import math
import xgboost as xgb
import matplotlib
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM

import warnings
warnings.simplefilter('ignore')



## Problem Formulation: Time Series Forecasting

Given a taxi stand and a 30-min interval, we have to predict pickups.
* (a): Use taxi stand as the center of region
* (b): Every stand in Porto Porto has to be broken up into 30-min  interval (set a parameter).<br>

Now, every row pickup has longitude and latitude in it, and we will use this to define pickup locations. 
We already know, about the pickup at time 't', we will predict the pickup at time 't+1' in the same region. Hence, this problem can be thought of as a 'Time Series Prediction' problem. It is a special case of regression problems. In short, we will use the data at time 't' to predict for time 't+1'.

## Performance Metric
* 1) Synmetric Mean Absolute Percentage Error(sMAPE)
* 2) Mean Squared Error(MSE)

# 2. Data Preparation

In [ ]:
# 1. Get the dataframe which includes only required colums.
# 2. Add time_bin (index of 10min intravel - use a parameter to which that trip belongs to).
# 3. Group by data, based on 'pickup_cluster' and 'time_bin'
startTime = datetime.now()
frame_data = read_csv("/home/tai/data/data_process.csv")

print("PREPARATION OF July 2013-June 2014 DATA.")
print("-"*35)

print("Number of columns = "+str(len(frame_data.columns)))
print("-"*35)

print("Number of rows = " + str(len(frame_data)))
print("-"*35)

In [ ]:
coord = frame_data[["LATITUDE", "LONGITUDE"]].values

In [ ]:
frame_data.head()

In [ ]:
minlat = frame_data['LATITUDE'].min()
maxlat = frame_data['LATITUDE'].max()
minlon = frame_data['LONGITUDE'].min()
maxlon = frame_data['LONGITUDE'].max()

In [ ]:
print("Min latitude: "+str(minlat))
print("Max latitude: "+str(maxlat))
print("Min LONGITUDE: "+str(minlon))
print("Max LONGITUDE: "+str(maxlon))

In [ ]:
coord=frame_data[['LATITUDE','LONGITUDE']].values

In [ ]:
coord

In [ ]:
len(coord)

In [ ]:
type(coord)

In [ ]:
#labels
cluster_labels=frame_data['STAND'].values

In [ ]:
taxistand=read_csv("/home/tai/data/taxistand.csv")

In [ ]:
taxistand

#### Plotting stand on the map

https://www.flickr.com/places/info/2459115 

In [ ]:
centerOfRegions = taxistand[['Latitude','Longitude']].values
noOfClusters = len(centerOfRegions)

In [ ]:
len(centerOfRegions)

In [ ]:
m = folium.Map(location = [41.161488, -8.628821], zoom_start=13)
for i in range(noOfClusters):
    folium.Marker([centerOfRegions[i][0], centerOfRegions[i][1]], popup = (str(np.round(centerOfRegions[i][0], 2))+", "+str(np.round(centerOfRegions[i][1], 2)))).add_to(m)
m

In [ ]:
m = folium.Map(location = [41.14961,-8.61099], zoom_start=13)
for i in range(noOfClusters):
    folium.Marker([centerOfRegions[i][0], centerOfRegions[i][1]], popup = taxistand.loc[i,'Descricao']).add_to(m)
m


#### Plotting Regions in Porto

In [ ]:
Porto_latitude_range = (41.0011, 41.2366)
Porto_Longitude_range = (-8.7891, -7.8751)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.scatter(x = frame_data['LONGITUDE'].values, y = frame_data['LATITUDE'].values, c = frame_data['STAND'].values, cmap = "Paired", s = 5)
ax.set_xlim(-8.79, -7.87)
ax.set_ylim(41.01, 41.24)
ax.set_title("Regions in Porto City")
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

plt.show()
#Longitude values vary from left to right i.e., horizontally
#Latitude values vary from top to bottom means i.e., vertically

In [ ]:
#Print number of items in taxi stand
count_points = frame_data.groupby(['STAND'])['STAND'].count()
point_count = count_points.values

In [ ]:
for i in range (len(point_count)):
    print(point_count[i])

The Campanhã Railway Station is a 19th-century railway station in the civil parish of Campanhã, in the municipality of Porto, district of Porto. It is the busiest taxistand in Porto as well.

In [ ]:
busiest=point_count.argmax()
taxistand.iloc[busiest]

In [ ]:
centerOfRegions

In [ ]:
centroids=centerOfRegions

#Calculate new average distance from points to their centroids
def k_mean_distance_new(data, cx, cy, i_centroid, cluster_labels):
        distances = [gpxpy.geo.haversine_distance(cx,cy,x,y) for (x, y) in data[cluster_labels == i_centroid]]
        return distances


distances_new = []
index=[]
for i, (cx, cy) in enumerate(centroids):
    if (i in count_points_new):
        mean_distance_new = k_mean_distance_new(coord, cx, cy, i, cluster_labels)
        distances_new.append(mean_distance_new)
        index.append(i)

In [ ]:
for i in range(0,63):
    plt.figure(figsize=(8,7))
    plt.scatter(x = frame_data['LONGITUDE'][frame_data['STAND'].values==i].values, y = frame_data['LATITUDE'][frame_data['STAND'].values==i].values, cmap = "Paired", s = 5,alpha=0.7)
    plt.xlim(-8.79, -7.87)
    plt.ylim(41.01, 41.24)
    plt.title("Stand " + str(i+1))
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.show()   

## Time Binning

In [ ]:
# 1372636800 : 2013-07-01 00:00:00   (Equivalent unix time)
# 1404172799 : 2014-06-30 00:00:00   (Equivalent unix time)

def pickup_bins(dataframe, year, granularity):
    pickupTime = dataframe["TIMESTAMP"].values
    unixTime = [1372636800, 1404172799]
    unix_year = unixTime[year-2013]
    time_bin = [int((i - unix_year)/(granularity*60)) for i in pickupTime]
    dataframe["TIME_BIN"] = np.array(time_bin)
    return dataframe

In [ ]:
porto_data = pickup_bins(frame_data, 2013,30)
print("Pickup time bins are assigned")
print("-"*35)

In [ ]:
porto_data.head()

In [ ]:
print("Number of time bin: ", str(len(np.unique(porto_data["TIME_BIN"]))))

In [ ]:
porto_data_timeBin_groupBy = porto_data[["STAND", "TIME_BIN", "CALL_TYPE"]].groupby(by = ["STAND", "TIME_BIN"]).count()
print("Stand and time bins are grouped.")
print("-"*35)


In [ ]:
porto_data_timeBin_groupBy.head()

## EXtracting more temporal features from the dataset

For each trip we will extract the weekday and the hour of pickup

In [ ]:
frame_data.shape

In [ ]:
from datetime import datetime
#convert timestamp to datetime in YMDHMS column
pickupTime = porto_data["TIMESTAMP"].values
YMDHMS = [datetime.fromtimestamp(i).strftime('%Y-%m-%d-%H-%M-%S') for i in pickupTime]
porto_data["Pickup_YMDHMS"] = np.array(YMDHMS)

In [ ]:
#Exctract pickup hour
pu_hour = [int(i.split('-')[3]) for i in YMDHMS]
porto_data["Pickup_hour"] = np.array(pu_hour)

In [ ]:
#extract pickup weekday for each trip
pu_weekday=[datetime.strptime(str(i),'%Y-%m-%d-%H-%M-%S').strftime('%A') for i in YMDHMS]
porto_data["Pickup_weekday"] = np.array(pu_weekday)

In [ ]:
porto_data.head(10)

In [ ]:
from sqlalchemy import create_engine

porto_db = create_engine('sqlite:///porto_db.db')

In [ ]:
import sqlite3
con = sqlite3.connect('C:\\Users\\Bahman\\Traffic Data - LUH\\Thesis Codes\\porto_db.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
con.close()

In [ ]:
#porto_data.to_sql('taxidata_table_record', porto_db, if_exists='append') 

In [ ]:
#ddd = pd.read_sql_query('SELECT * FROM taxidata_table_record ', porto_db)

In [ ]:
df_pu = pd.read_sql_query('SELECT Pickup_hour AS time, count(*) AS PUcount \
                        FROM taxidata_table_record \
                        GROUP BY pickup_hour', porto_db)

In [ ]:
df_pu.head()

In [ ]:
df_pickup_weekday = pd.read_sql_query('SELECT Pickup_weekday, count(*) AS PUcount \
                        FROM taxidata_table_record \
                        GROUP BY pickup_weekday', porto_db)

In [ ]:
df_pickup_weekday

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(7,7))
objects = df_pickup_weekday['Pickup_weekday']
y_pos = np.arange(len(objects))
x_pos = df_pickup_weekday['PUcount']
 
plt.bar(y_pos, x_pos, align='center', alpha=1.0)
plt.xticks(y_pos, objects)
plt.ylabel('Number of Pickups')
plt.xlabel('weekdays')

plt.title('Number of pickups during the week')
 
plt.show()

## Smoothing

In [ ]:
porto_data.head(1)

In [ ]:
porto_data.tail(1)

In [ ]:
# Gets the unique time bins where pickup values are present for each stand.

def getUniqueBinsWithPickups(dataframe):
    values = []
    for i in range(63):          #we have total 63 clusters
        stand_id = dataframe[dataframe["STAND"] == (i+1)]
        unique_stand_id = list(set(stand_id["TIME_BIN"]))
        unique_stand_id.sort()   #inplace sorting
        values.append(unique_stand_id)
    return values

In [ ]:
# Fill the missing value with 0's.
def fillMissingWithZero(numberOfPickups, correspondingTimeBin):
    ind = 0
    smoothed_regions = []
    for c in range(0, 63):
        smoothed_bins = []
        for t in range(17519):    #there are total 17519 time bins
            if t in correspondingTimeBin[c]:   #if a time bin is present in "correspondingTimeBin" in stand 'c', 
            #then it means there is a pickup, in this case, we are simply adding number of pickups, else we are adding 0.
                smoothed_bins.append(numberOfPickups[ind])
                ind += 1
            else:
                smoothed_bins.append(0)
        smoothed_regions.extend(smoothed_bins)
    return smoothed_regions

In [ ]:
def smoothing(numberOfPickups, correspondingTimeBin):
    ind = 0
    repeat = 0
    smoothed_region = []
    for cluster in range(0, 63):
        smoothed_bin = []
        for t1 in range(17519):
            if repeat != 0:   #this will ensure that we shall not fill the pickup values again which we already filled by smoothing
                repeat -= 1
            else:
                if t1 in correspondingTimeBin[cluster]:
                    smoothed_bin.append(numberOfPickups[ind])
                    ind += 1
                else:
                    if t1 == 0:           
    #<---------------------CASE-1:Pickups missing in the beginning------------------------>
                        for t2 in range(t1, 17519):
                            if t2 not in correspondingTimeBin[cluster]:
                                continue
                            else:
                                right_hand_limit = t2
                                smoothed_value = (numberOfPickups[ind]*1.0)/((right_hand_limit + 1)*1.0)
                                for i in range(right_hand_limit + 1):
                                    smoothed_bin.append(math.ceil(smoothed_value))
                                ind += 1
                                repeat = right_hand_limit - t1
                                
                    if t1 != 0:
                        right_hand_limit = 0
                        for t2 in range(t1, 17519):
                            if t2 not in correspondingTimeBin[cluster]:
                                continue
                            else:
                                right_hand_limit = t2
                                break
                        if right_hand_limit == 0:
    #<---------------------CASE-2: Pickups MISSING IN THE END------------------------------>
                            smoothed_value = (numberOfPickups[ind-1]*1.0)/(((4464 - t1)+1)*1.0)
                            del smoothed_bin[-1]
                            for i in range((17519 - t1)+1):
                                smoothed_bin.append(math.ceil(smoothed_value))
                            repeat = (17519 - t1) - 1    
    #<---------------------CASE-3: Pickups MISSING IN MIDDLE OF TWO VALUES----------------> 
                        else: 
                            smoothed_value = ((numberOfPickups[ind-1] + numberOfPickups[ind])*1.0)/(((right_hand_limit - t1)+2)*1.0)
                            del smoothed_bin[-1]
                            for i in range((right_hand_limit - t1)+2):
                                smoothed_bin.append(math.ceil(smoothed_value))
                            ind += 1
                            repeat = right_hand_limit - t1                        
        smoothed_region.extend(smoothed_bin)
    return smoothed_region

# when we multiply any integer with "1.0", then it will be converted into float.

In [ ]:
def countZeros(num):
    count = 0
    for i in num:
        if i == 0:
            count += 1
    return count

In [ ]:
unique_binswithPickup_porto_data = getUniqueBinsWithPickups(porto_data)

In [ ]:
#unique_binswithPickup_porto_data[0]

In [ ]:
porto_data_fillZero = fillMissingWithZero(porto_data_timeBin_groupBy["CALL_TYPE"].values, unique_binswithPickup_porto_data)

In [ ]:
regionWisePickup_porto_data = []
for i in range(63):
    regionWisePickup_porto_data.append(porto_data_fillZero[17519*i:((17519*i)+17519)])

In [ ]:
print(len(regionWisePickup_porto_data))
print(len(regionWisePickup_porto_data[0]))

In [ ]:
len(regionWisePickup_porto_data[62])

### POI Dataset

In [ ]:
df_poidata=pd.read_csv("/home/tai/data/poi_populartimes.csv")

In [ ]:
poi=pd.merge(df_poidata, taxistand, left_on='nearest_taxi_stand', right_on='Descricao', how='left')

In [ ]:
poi.iloc[0]

In [ ]:
poi.iloc[0]['Latitude']

In [ ]:
poi.loc[0,'ID']

In [ ]:
a=eval(poi.loc[0,'populartimes'])

In [ ]:
str(a)

In [ ]:
for i in range(len(a)):
    print(a[i]['data'])

In [ ]:
print(a[0]['data'][2:])
for i in range(len(a)-2):
    print(a[i+1]['data'])
print(a[6]['data'][:]+a[0]['data'][0:2])


In [ ]:
df_poidata.tail(2)

In [ ]:
poi.head(1)

In [ ]:
frame_data.head(1)

In [ ]:
frame_data.tail(1)

The first pickup date/time: 2013-07-01 at 02-00-53 

The last pickup date/time:2014-07-01 at 01-59-14

Regarding to start/end pickup date/time we need to vectorize the populartimes in a time series with the same timestamp.

We form standvisit matrix(63,17520) which indicates 63 timeseries for 63 taxistand. Each row shows the number of visitors around a taxi stand. In this case we aggregated populartimes of poi's. The radius is the distance between current taxistand to the nearest taxistand.

In [ ]:
standvisit=np.zeros((63,17519 ))
for p in range(len(poi)):
    a=eval(poi.loc[p,'populartimes'])
    visitlist=[]
    
    #visitlist should be started from Monday at 2:00 am and ended on Monday at 2:00 am
    visitlist = a[0]['data'][2:]
    for i in range(len(a)-2):
        visitlist=visitlist+a[i+1]['data']
    visitlist=visitlist+a[6]['data'][:]+a[0]['data'][0:2]
    #we need visit data for the whole year 
    visitlist=visitlist*52
    visitlist=visitlist+a[0]['data'][2:]+a[1]['data'][0:2]
    #Since timestamp is 30 minute, we douplicate the visitlist
    visitlist=np.repeat(visitlist, 2)
    #The length of pickup timeseries is 17519, so we have to remove tha last 30-min timestamp from visitlist.
    visitlist=visitlist[:17519]
    #Add a new column to the poi dataset which stotes the visitlist for each place during the year
    poi.loc[p,'visitors'] =str(visitlist)
    #Agreggate the number of visits around each taxi stand. We calculate the total nymber of visits around a specific taxistand
    s=poi.loc[p,'ID']
    standvisit[s-1]=np.add(standvisit[s-1],visitlist)
    #visitlist.clear()
    #poi.apply(lambda s: eval(poi['visitors']), axis=1)
    #arr.toarray().tolist()
    #poi.groupby(['ID','Descricao'])['visitors'].sum()

In [ ]:
len(visitlist)

In [ ]:
b=poi.loc[0,'visitors']

In [ ]:
poi.head(1)

In [ ]:
taxistand.iloc[1]

In [ ]:
del poi['visittimes']

In [ ]:
poi.head(1)

In [ ]:
#taxistand

In [ ]:
#taxistand.groupby(['Descricao'])['Descricao'].count()

In [ ]:
len(poi)

In [ ]:
poi.tail(1)

In [ ]:
# poi time series for each taxi stand
standvisit.shape

In [ ]:
standvisit

### Assigning POI to the closest taxi stands

POI Dataset: This time we assign each pio to the nearest tasxistand, which mean we somehow define a zone nearby each taxistand. 
    In D2 dataset, we assign each trip to the closet taxi stand, we would like to use the same idea for pio.

In [ ]:
#There is no redundancy, which means we have 2051 distinct place in Porto
poi['id'].nunique()

In [ ]:
a=poi.groupby('ID')['ID'].count()

In [ ]:
#a
#np.sum(a)

In [ ]:
#poi.iloc[0]

In [ ]:
taxistand.head(1)

In [ ]:
from haversine import haversine
from numpy import zeros
dist = np.zeros((p,t))
#dist is a 2051 * 63 matrix
p=len(poi)
t=len(taxistand)
for plc in range(len(poi)):
    slat=poi.loc[plc,"latitude"]
    slon=poi.loc[plc,"langitude"]
    loc1 = (slat, slon)
    for txs in range(len(taxistand)):
        elat=taxistand.loc[txs,"Latitude"]
        elon=taxistand.loc[txs,"Longitude"]
        loc2 = (elat, elon)
        dist[plc][txs]=haversine(loc1, loc2)*1000

In [ ]:
dist=np.array(np.round(dist), dtype=int)

In [ ]:
dist

In [ ]:
#poiZoneNumber is the closest taxi stand to a poi
poiZoneNumber=np.zeros([2051,2])
for d in range(len(dist)):
    lst=dist[d,:]
    poiZoneNumber[d][0]=int(np.min(lst))
    poiZoneNumber[d][1]=np.argmin(lst)
    #print(d,np.min(lst),np.argmin(lst))
   

In [ ]:
poiZoneNumber=np.array(np.round(poiZoneNumber), dtype=int)

In [ ]:
poiZoneNumber=pd.DataFrame(poiZoneNumber)

In [ ]:
poiZoneNumber.columns = ['dist2stand', 'poistand']

In [ ]:
poiZoneNumber.shape

In [ ]:
poiZoneNumber.head(2)

In [ ]:
poi.head(1)

In [ ]:
poidata=pd.concat([poi, poiZoneNumber], axis=1)

In [ ]:
# ID	Descricao	Latitude	Longitude are the features of nearest taxi stand when we assume that radious is the distance from target stand to the closest stand.
# dist2stand	poistand are the features for nearest taxi stand when we assign each poi to the closest taxistand based on the distance.

#Unnamed: 0	id	name	address	types	coordinates	rating	rating_n	international_phone_number	populartimes	...	nearest_taxi_stand	visittimes	latitude	langitude are the features we extract from google map for each poi
poidata.head(2)

In [ ]:
type(poidata)

### Use this for prediction, by aggregrating all POIs assigned to taxi stand

In [ ]:
newstandvisit=np.zeros((63,17519 ))
for p in range(len(poidata)):
    a=eval(poidata.loc[p,'populartimes'])
    visitlist=[]
    
    #visitlist should be started from Monday at 2:00 am and ended on Monday at 2:00 am
    visitlist = a[0]['data'][2:]
    for i in range(len(a)-2):
        visitlist=visitlist+a[i+1]['data']
    visitlist=visitlist+a[6]['data'][:]+a[0]['data'][0:2]
    #we need visit data for the whole year 
    visitlist=visitlist*52
    visitlist=visitlist+a[0]['data'][2:]+a[1]['data'][0:2]
    #Since timestamp is 30 minute, we douplicate the visitlist
    visitlist=np.repeat(visitlist, 2)
    #The length of pickup timeseries is 17519, so we have to remove tha last 30-min timestamp from visitlist.
    visitlist=visitlist[:17519]
    #Add a new column to the poi dataset which stotes the visitlist for each place during the year
    poidata.loc[p,'newvisitors'] =str(visitlist)
    #Agreggate the number of visits around each taxi stand. We calculate the total nymber of visits around a specific taxistand
    s=poidata.loc[p,'poistand']
    newstandvisit[s-1]=np.add(newstandvisit[s-1],visitlist)
    #visitlist.clear()
    #poi.apply(lambda s: eval(poi['visitors']), axis=1)
    #arr.toarray().tolist()
    #poi.groupby(['ID','Descricao'])['visitors'].sum()

In [ ]:
newtotalvisit=np.sum(newstandvisit,axis=1,dtype=int)

In [ ]:
len(newtotalvisit)

In [ ]:
newtotalvisit

In [ ]:
totalvisit=np.sum(standvisit,axis=1,dtype=int)

In [ ]:
len(totalvisit)

In [ ]:
totalvisit

# 3. Regression Models

### Preparing Data

Preparing data to be split into train and test, The below code prepares data in cumulative form which will be later split into
test and train

There are total 63 clusters and for the dataset and there are total 17519 time bins. 
For each stand there are 17519 time bins and so, for 63 clusters there will be 17519*63 pickup values because after 
smoothing each time bin has pickup.
We will have a total of 17519x63 = 1.103.697 pickup values.

In [ ]:
# we take number of pickups that are happened in last 10 30min intravels
number_of_time_stamps = 10
# TruePickups varaible
# it is list of lists
# It will be used as true labels/ground truth. Now since we are taking previous 10 pickups as a training data for predicting
# next pickup(here next pickup will be a true/ground truth pickup), so "TruePickups" will not contain first five pickups of each 
# cluster. It will contain number of pickups 17519-10 = 17509 for each cluster. 
TruePickups = []

# lat,lon will contain 17519-10=17509 times latitude of cluster center for every stand
# it is list of lists
lat = []
lon = []
# we will code each day 
# sunday = 0, monday=1, tue = 2, wed=3, thur=4, fri=5,sat=6
day_of_week = []

feat = []
numberofstands = 63
lengthoftimeseries= len(regionWisePickup_porto_data[0])

feat = [0]*number_of_time_stamps
for i in range(numberofstands):
        lat.append([centerOfRegions[i][0]]*(lengthoftimeseries-number_of_time_stamps)) 
        lon.append([centerOfRegions[i][1]]*(lengthoftimeseries-number_of_time_stamps))
        day_of_week.append([int(((int(j/48)%7)+number_of_time_stamps)%7) for j in range(number_of_time_stamps, lengthoftimeseries)])
        #48 is the time bin for a day (30 min)
        feat = np.vstack((feat, [regionWisePickup_porto_data[i][k:k+number_of_time_stamps] for k in range(0, len(regionWisePickup_porto_data[i]) - (number_of_time_stamps))]))
        #feat = np.vstack(([regionWisePickup_porto_data[i][k:k+number_of_time_stamps] for k in range(0, len(regionWisePickup_porto_data[i]) - (number_of_time_stamps))]))
        TruePickups.append(regionWisePickup_porto_data[i][number_of_time_stamps:])
        #output contains pickup values of all the regions and of each time stamp, except first 5 time stamp pickups of each region.
feat = feat[1:]

In [ ]:
len(lon)

In [ ]:
len(lon[0])*len(lon)

In [ ]:
len(day_of_week[0])*len(day_of_week)

In [ ]:
len(lat[0])*len(lat) == len(lon[0])*len(lon) == len(day_of_week[0])*len(day_of_week) == 17509*63 == len(feat) == len(TruePickups[0])*len(TruePickups)

In [ ]:
feat

In [ ]:
len(feat)

### Adding Predictions of Weighted Moving Average Predictions as a feature in our data

Getting the predictions of weighted moving averages to be used as a feature in cumulative form.

Upto now we computed 8 features for every data point that starts from 50th min of the day.
1. cluster center latitude
2. cluster center longitude
3. day of the week 
4. f_t_1: number of pickups that are happened previous t-1st 10min interval
5. f_t_2: number of pickups that are happened previous t-2nd 10min interval
6. f_t_3: number of pickups that are happened previous t-3rd 10min interval
7. f_t_4: number of pickups that are happened previous t-4th 10min interval
8. f_t_5: number of pickups that are happened previous t-5th 10min interval

From the baseline models we said that the weighted moving avarage predictions gives us the best error.
We will try to add the same weighted moving avarage predictions at time t as a feature to our data.<br>
Weighted Moving Average -> $P_{t} = ( N*P_{t-1} + (N-1)*P_{t-2} + (N-2)*P_{t-3} .... 1*P_{t-n} )/(N*(N+1)/2)$

In [ ]:
# "predicted_pickup_values": it is a temporary array that store weighted moving avarag prediction values for each 30min intervl, 
# for each cluster it will get reset.
# for every cluster it contains 17519 values
predicted_pickup_values = []

# "predicted_pickup_values_list"
# it is list of lists
predicted_pickup_values_list = []

predicted_value = -1  #it will contain cuurent predicted_value. Default is given -1 which will be replaced later

window_size = 2
for i in range(numberofstands):
    for j in range(lengthoftimeseries):
        if j == 0:
            predicted_value = regionWisePickup_porto_data[i][j]
            predicted_pickup_values.append(0)
        else:
            if j>=window_size:
                sumPickups = 0
                sumOfWeights = 0
                for k in range(window_size, 0, -1):
                    sumPickups += k*(regionWisePickup_porto_data[i][j -window_size + (k - 1)])
                    sumOfWeights += k
                predicted_value = int(sumPickups/sumOfWeights)
                predicted_pickup_values.append(predicted_value)
            else:
                sumPickups = 0
                sumOfWeights = 0
                for k in range(j, 0, -1):
                    sumPickups += k*regionWisePickup_porto_data[i][k-1]
                    sumOfWeights += k
                predicted_value = int(sumPickups/sumOfWeights)
                predicted_pickup_values.append(predicted_value)
                
    predicted_pickup_values_list.append(predicted_pickup_values[10:])
    predicted_pickup_values = []

In [ ]:
len(predicted_pickup_values_list)

In [ ]:
len(predicted_pickup_values_list[0])

In [ ]:
#%matplotlib notebook
from matplotlib.backends.backend_pdf import PdfPages
pp = PdfPages('stand_pattern.pdf')
for i in range(63):#number of cluster
    fig = plt.figure(figsize = (18, 6))
    plt.plot(regionWisePickup_porto_data[i][:336])
    plt.title("Pickup pattern of stand "+str(i+1)+" for 7 days (01/07/2013-07/07/2013)")
    plt.xlabel("30 Minute Time Bins")
    plt.ylabel("Number of Pickups")
    pp.savefig(fig)
pp.close()

## Data Preparation for regression models
Before we start predictions using the tree based regression models we take Jan 2016 pickup data and split it such that for every region we have 70% data in train and 30% in test, ordered date-wise for every region.

In [ ]:
print("size of total train data :" +str(int(17509*63*0.7)))
print("size of total test data :" +str(int(17509*63*0.3)))
trainsize =int((lengthoftimeseries - number_of_time_stamps)*numberofstands*0.7)
testsize=int((lengthoftimeseries - number_of_time_stamps)*numberofstands*0.3)


In [ ]:
trainsize


In [ ]:
testsize

In [ ]:
print("size of train data for one stand:" +str(int(17509*0.7)))
print("size of total test data for one stand:" +str(int(17509*0.3)))
trainsizeonestand=int((lengthoftimeseries - number_of_time_stamps)*0.7)
testsizeonestand=int((lengthoftimeseries - number_of_time_stamps)*0.3)

In [ ]:
trainsizeonestand

In [ ]:
testsizeonestand

In [ ]:
train_previousFive_pickups  = [feat[(i*(lengthoftimeseries - number_of_time_stamps)):((lengthoftimeseries - number_of_time_stamps)*i+trainsizeonestand)] for i in range(numberofstands)]
test_previousFive_pickups  = [feat[(i*(lengthoftimeseries - number_of_time_stamps)+trainsizeonestand):((lengthoftimeseries - number_of_time_stamps)*(i+1))] for i in range(numberofstands)]

In [ ]:
print("Train Data: Total number of clusters = {}. Number of points in each cluster = {}. Total number of training points = {}".format(len(train_previousFive_pickups), len(train_previousFive_pickups[0]), len(train_previousFive_pickups)*len(train_previousFive_pickups[0])))
print("Test Data: Total number of clusters = {}. Number of points in each cluster = {}. Total number of test points = {}".format(len(test_previousFive_pickups), len(test_previousFive_pickups[0]), len(test_previousFive_pickups)*len(test_previousFive_pickups[0])))

In [ ]:
#taking 70% data as train data from each cluster
train_lat = [i[:trainsizeonestand] for i in lat]
train_lon = [i[:trainsizeonestand] for i in lon]
train_weekDay = [i[:trainsizeonestand] for i in day_of_week]
train_weighted_avg = [i[:trainsizeonestand] for i in predicted_pickup_values_list]
train_TruePickups = [i[:trainsizeonestand] for i in TruePickups]

In [ ]:
#taking 30% data as test data from each cluster
test_lat = [i[trainsizeonestand:] for i in lat]
test_lon = [i[trainsizeonestand:] for i in lon]
test_weekDay = [i[trainsizeonestand:] for i in day_of_week]
test_weighted_avg = [i[trainsizeonestand:] for i in predicted_pickup_values_list]
test_TruePickups = [i[trainsizeonestand:] for i in TruePickups]

In [ ]:
# convert from lists of lists of list to lists of list
train_pickups = []
test_pickups = []
for i in range(numberofstands):
    train_pickups.extend(train_previousFive_pickups[i])
    test_pickups.extend(test_previousFive_pickups[i])    

In [ ]:
#stacking pickups,frequencies and amplitudes horizontally.
# a = [1,2,3,4,5]
# b = [6,7,8,9,10]
# c = [11,12,13,14,15]
# d = np.hstack((a, b, c))
# d = array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

#train_prevPickups_freq_amp = np.hstack((train_pickups, train_freq, train_amp))
#test_prevPickups_freq_amp = np.hstack((test_pickups, test_freq, test_amp))
#Khong su dung thong tin them
train_prevPickups_freq_amp = train_pickups
test_prevPickups_freq_amp = test_pickups


In [ ]:
print("Number of data points in train data = {}. Number of columns till now = {}".format(len(train_prevPickups_freq_amp), len(train_prevPickups_freq_amp[0])))
print("Number of data points in test data = {}. Number of columns till now = {}".format(len(test_prevPickups_freq_amp), len(test_prevPickups_freq_amp[0])))

In [ ]:
# converting lists of lists into single list i.e flatten
# a  = [[1,2,3,4],[4,6,7,8]]
# print(sum(a,[]))
# [1, 2, 3, 4, 4, 6, 7, 8]

train_flat_lat = sum(train_lat, [])
train_flat_lon = sum(train_lon, [])
train_flat_weekDay = sum(train_weekDay, [])
train_weighted_avg_flat = sum(train_weighted_avg, [])
train_TruePickups_flat = sum(train_TruePickups, [])

test_flat_lat = sum(test_lat, [])
test_flat_lon = sum(test_lon, [])
test_flat_weekDay = sum(test_weekDay, [])
test_weighted_avg_flat = sum(test_weighted_avg, [])
test_TruePickups_flat = sum(test_TruePickups, [])

In [ ]:
len(train_prevPickups_freq_amp)

In [ ]:
len(test_TruePickups_flat)

In [ ]:
train_prevPickups_freq_amp[0]

In [ ]:
#Visualize data
fig_pickup = plt.figure(figsize = (8, 6))
plt.plot(train_TruePickups_flat[0:50])
plt.title("Pickup Pattern for Jan-2016.")
plt.xlabel("30 Minute Time Bins")
plt.ylabel("Number of Pickups")

In [ ]:
#train dataframe
#columns = ['ft_10','ft_9','ft_8','ft_7','ft_6','ft_5','ft_4','ft_3','ft_2','ft_1', 'freq1', 'freq2','freq3','freq4','freq5', 'Amp1', 'Amp2', 'Amp3', 'Amp4', 'Amp5']
columns = ['ft_10','ft_9','ft_8','ft_7','ft_6','ft_5','ft_4','ft_3','ft_2','ft_1']
Train_DF = pd.DataFrame(data = train_prevPickups_freq_amp, columns = columns)
#Train_DF["Latitude"] = train_flat_lat
#Train_DF["Longitude"] = train_flat_lon
#Train_DF["WeekDay"] = train_flat_weekDay
#Train_DF["WeightedAvg"] = train_weighted_avg_flat

In [ ]:
#test dataframe
Test_DF = pd.DataFrame(data = test_prevPickups_freq_amp, columns = columns)
#Test_DF["Latitude"] = test_flat_lat
#Test_DF["Longitude"] = test_flat_lon
#Test_DF["WeekDay"] = test_flat_weekDay
#Test_DF["WeightedAvg"] = test_weighted_avg_flat

In [ ]:
print("Shape of train data = "+str(Train_DF.shape))
print("Shape of test data = "+str(Test_DF.shape))

In [ ]:
Train_DF.head()

In [ ]:
Test_DF.head()

In [ ]:
#Define SMAPE
#{\displaystyle {\text{SMAPE}}={\frac {100\%}{n}}\sum _{t=1}^{n}{\frac {|F_{t}-A_{t}|}{|A_{t}|+|F_{t}|+c}}}
#in our case c=1, use laplace correction
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)+1) / 100.0
    diff = np.abs(y_true - y_pred) / denominator
    #diff[denominator == 0] = 1
    return np.nanmean(diff)

## Linear Regression function

In [ ]:
def lin_regression(train_data, train_true, test_data, test_true):
    
    #standardizing the data
    train_std = StandardScaler().fit_transform(train_data)
    test_std = StandardScaler().fit_transform(test_data)
    
    #hyper-paramater tuning
    clf = SGDRegressor(loss = "squared_loss", penalty = "l2")
    values = [10**-14, 10**-12, 10**-10, 10**-8, 10**-6, 10**-4, 10**-2, 10**0, 10**2, 10**4, 10**6]
    hyper_parameter = {"alpha": values}
    
    best_parameter = GridSearchCV(clf, hyper_parameter, scoring = "neg_mean_absolute_error", cv = 3)
    best_parameter.fit(train_std, train_true)
    alpha = best_parameter.best_params_["alpha"]
    
    #applying linear regression with best hyper-parameter
    clf = SGDRegressor(loss = "squared_loss", penalty = "l2", alpha = alpha)
    clf.fit(train_std, train_true)
    train_pred = clf.predict(train_std)
    train_MAPE = mean_absolute_error(train_true, train_pred)/ (sum(train_true)/len(train_true))
    train_SMAPE = smape(train_true, train_pred)
    train_MSE = mean_squared_error(train_true, train_pred)
    train_RMSE = math.sqrt(train_MSE) ## bonus
    test_pred = clf.predict(test_std)
    test_MAPE = mean_absolute_error(test_true, test_pred)/ (sum(test_true)/len(test_true))
    test_SMAPE = smape(test_true, test_pred)
    test_MSE = mean_squared_error(test_true, test_pred)
    test_RMSE = math.sqrt(test_MSE)
    return train_MAPE, train_MSE, train_RMSE, train_SMAPE, test_MAPE, test_MSE, test_RMSE, test_SMAPE, alpha

## Random Forest Regressor function

In [ ]:
def randomFor(train_data, train_true, test_data, test_true):
    
    #hyper-paramater tuning
    values = [10, 40, 80, 150, 600, 800]
    clf = RandomForestRegressor(n_jobs = -1)
    hyper_parameter = {"n_estimators": values}
    ##
    hyper_parameter
    ##
    best_parameter = GridSearchCV(clf, hyper_parameter, scoring = "neg_mean_absolute_error", cv = 3)
    best_parameter.fit(train_data, train_true)
    estimators = best_parameter.best_params_["n_estimators"]
    
    #applying random forest with best hyper-parameter
    clf = RandomForestRegressor(n_estimators = estimators, n_jobs = -1)
    clf.fit(train_data, train_true)
    train_pred = clf.predict(train_data)
    train_MAPE = mean_absolute_error(train_true, train_pred)/ (sum(train_true)/len(train_true))
    train_SMAPE = smape(train_true, train_pred)
    train_MSE = mean_squared_error(train_true, train_pred)
    train_RMSE = math.sqrt(train_MSE)
    test_pred = clf.predict(test_data)
    test_MAPE = mean_absolute_error(test_true, test_pred)/ (sum(test_true)/len(test_true))
    test_SMAPE = smape(test_true, test_pred)
    test_MSE = mean_squared_error(test_true, test_pred)
    test_RMSE = math.sqrt(test_MSE)
    return train_MAPE, train_MSE, train_RMSE, train_SMAPE, test_MAPE, test_MSE, test_RMSE, test_SMAPE,estimators

## XGBoost Regressor function

In [ ]:
def xgboost_reg(train_data, train_true, test_data, test_true):
    #hyper-parameter tuning
    hyper_parameter = {"max_depth":[1, 2, 3, 4], "n_estimators":[40, 80, 150, 600]}
    clf = xgb.XGBRegressor()
    best_parameter = GridSearchCV(clf, hyper_parameter, scoring = "neg_mean_absolute_error", cv = 3)
    best_parameter.fit(train_data, train_true)
    estimators = best_parameter.best_params_["n_estimators"]
    depth = best_parameter.best_params_["max_depth"]
    
    #applying xgboost regressor with best hyper-parameter
    clf = xgb.XGBRegressor(max_depth = depth, n_estimators = estimators)
    clf.fit(train_data, train_true)
    train_pred = clf.predict(train_data)
    train_MAPE = mean_absolute_error(train_true, train_pred)/ (sum(train_true)/len(train_true))
    train_SMAPE = smape(train_true, train_pred)
    train_MSE = mean_squared_error(train_true, train_pred)
    train_RMSE = math.sqrt(train_MSE)
    test_pred = clf.predict(test_data)
    test_MAPE = mean_absolute_error(test_true, test_pred)/ (sum(test_true)/len(test_true))
    test_SMAPE = smape(test_true, test_pred)
    test_MSE = mean_squared_error(test_true, test_pred)
    test_RMSE = math.sqrt(test_MSE)
    return train_MAPE, train_MSE, train_RMSE, train_SMAPE, test_MAPE, test_MSE, test_RMSE,test_SMAPE,depth,estimators

In [ ]:
trainMAPE_lr, trainMSE_lr, train_RMSE_lr, train_SMAPE_lr,testMAPE_lr, testMSE_lr, test_RMSE_lr,test_SMAPE_lr = lin_regression(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)
trainMAPE_rf, trainMSE_rf, train_RMSE_rf, train_SMAPE_rf,testMAPE_rf, testMSE_rf, test_RMSE_rf,test_SMAPE_rf = randomFor(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)
trainMAPE_xgb, trainMSE_xgb, train_RMSE_xgb, train_SMAPE_xgb,testMAPE_xgb, testMSE_xgb, test_RMSE_xgb,test_SMAPE_xgb = xgboost_reg(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)

# Model Comparison

In [ ]:
error_table_regressions = pd.DataFrame(columns = ["Model", "TrainMAPE(%)", "TrainMSE", "TrainRMSE","TrainSMAPE(%)","TestMAPE(%)", "TestMSE", "TestRMSE","TestSMAPE(%)"])

error_table_regressions = error_table_regressions.append(pd.DataFrame([["Linear Regression", trainMAPE_lr*100, trainMSE_lr, train_RMSE_lr, train_SMAPE_lr, testMAPE_lr*100, testMSE_lr,test_RMSE_lr,test_SMAPE_lr ]], columns = ["Model", "TrainMAPE(%)", "TrainMSE", "TrainRMSE","TrainSMAPE(%)","TestMAPE(%)", "TestMSE","TestRMSE","TestSMAPE(%)"]))
error_table_regressions = error_table_regressions.append(pd.DataFrame([["Random Forest Regression", trainMAPE_rf*100, trainMSE_rf, train_RMSE_rf, train_SMAPE_rf, testMAPE_rf*100, testMSE_rf, test_RMSE_rf, test_SMAPE_rf]], columns = ["Model", "TrainMAPE(%)", "TrainMSE","TrainRMSE","TrainSMAPE(%)", "TestMAPE(%)", "TestMSE","TestRMSE","TestSMAPE(%)"]))
error_table_regressions = error_table_regressions.append(pd.DataFrame([["XGBoost Regressor", trainMAPE_xgb*100, trainMSE_xgb,train_RMSE_xgb, train_SMAPE_xgb, testMAPE_xgb*100, testMSE_xgb,test_RMSE_xgb, test_SMAPE_xgb]], columns = ["Model", "TrainMAPE(%)", "TrainMSE", "TrainRMSE","TrainSMAPE(%)","TestMAPE(%)", "TestMSE","TestRMSE","TestSMAPE(%)"]))
error_table_regressions.reset_index(drop = True, inplace = True)

In [ ]:
error_table_regressions

# Finding parameter for model

In [ ]:
best_number_of_time_stamps_lr=2
min_sMAPE_lr=100

best_number_of_time_stamps_rf=2
min_sMAPE_rf=100

best_number_of_time_stamps_xgb=2
min_sMAPE_xgb=100


for number_of_time_stamps in range(2, 20):    
    # TruePickups varaible
    # it is list of lists
    # It will be used as true labels/ground truth. Now since we are taking previous 10 pickups as a training data for predicting
    # next pickup(here next pickup will be a true/ground truth pickup), so "TruePickups" will not contain first five pickups of each 
    # cluster. It will contain number of pickups 17519-10 = 17509 for each cluster. 
    TruePickups = []

    # lat,lon will contain 17519-10=17509 times latitude of cluster center for every stand
    # it is list of lists
    lat = []
    lon = []
    # we will code each day 
    # sunday = 0, monday=1, tue = 2, wed=3, thur=4, fri=5,sat=6
    day_of_week = []

    feat = []
    numberofstands = 1
    lengthoftimeseries= int(len(regionWisePickup_porto_data[0])/1)

    feat = [0]*number_of_time_stamps
    for i in range(numberofstands):
            lat.append([centerOfRegions[i][0]]*(lengthoftimeseries-number_of_time_stamps)) 
            lon.append([centerOfRegions[i][1]]*(lengthoftimeseries-number_of_time_stamps))
            day_of_week.append([int(((int(j/48)%7)+number_of_time_stamps)%7) for j in range(number_of_time_stamps, lengthoftimeseries)])
            #48 is the time bin for a day (30 min)
            feat = np.vstack((feat, [regionWisePickup_porto_data[i][k:k+number_of_time_stamps] for k in range(0, len(regionWisePickup_porto_data[i]) - (number_of_time_stamps))]))
            #feat = np.vstack(([regionWisePickup_porto_data[i][k:k+number_of_time_stamps] for k in range(0, len(regionWisePickup_porto_data[i]) - (number_of_time_stamps))]))
            TruePickups.append(regionWisePickup_porto_data[i][number_of_time_stamps:])
            #output contains pickup values of all the regions and of each time stamp, except first 5 time stamp pickups of each region.
    feat = feat[1:]
    # "predicted_pickup_values": it is a temporary array that store weighted moving avarag prediction values for each 30min intervl, 
    # for each cluster it will get reset.
    # for every cluster it contains 17519 values
    predicted_pickup_values = []

    # "predicted_pickup_values_list"
    # it is list of lists
    predicted_pickup_values_list = []

    predicted_value = -1  #it will contain cuurent predicted_value. Default is given -1 which will be replaced later

    window_size = 2
    for i in range(numberofstands):
        for j in range(lengthoftimeseries):
            if j == 0:
                predicted_value = regionWisePickup_porto_data[i][j]
                predicted_pickup_values.append(0)
            else:
                if j>=window_size:
                    sumPickups = 0
                    sumOfWeights = 0
                    for k in range(window_size, 0, -1):
                        sumPickups += k*(regionWisePickup_porto_data[i][j -window_size + (k - 1)])
                        sumOfWeights += k
                    predicted_value = int(sumPickups/sumOfWeights)
                    predicted_pickup_values.append(predicted_value)
                else:
                    sumPickups = 0
                    sumOfWeights = 0
                    for k in range(j, 0, -1):
                        sumPickups += k*regionWisePickup_porto_data[i][k-1]
                        sumOfWeights += k
                    predicted_value = int(sumPickups/sumOfWeights)
                    predicted_pickup_values.append(predicted_value)

        predicted_pickup_values_list.append(predicted_pickup_values[10:])
        predicted_pickup_values = []
    trainsize =int((lengthoftimeseries - number_of_time_stamps)*numberofstands*0.7)
    testsize=int((lengthoftimeseries - number_of_time_stamps)*numberofstands*0.3)
    trainsizeonestand=int((lengthoftimeseries - number_of_time_stamps)*0.7)
    testsizeonestand=int((lengthoftimeseries - number_of_time_stamps)*0.3)
    train_previousFive_pickups  = [feat[(i*(lengthoftimeseries - number_of_time_stamps)):((lengthoftimeseries - number_of_time_stamps)*i+trainsizeonestand)] for i in range(numberofstands)]
    test_previousFive_pickups  = [feat[(i*(lengthoftimeseries - number_of_time_stamps)+trainsizeonestand):((lengthoftimeseries - number_of_time_stamps)*(i+1))] for i in range(numberofstands)]
    print("Train Data: Total number of clusters = {}. Number of points in each cluster = {}. Total number of training points = {}".format(len(train_previousFive_pickups), len(train_previousFive_pickups[0]), len(train_previousFive_pickups)*len(train_previousFive_pickups[0])))
    print("Test Data: Total number of clusters = {}. Number of points in each cluster = {}. Total number of test points = {}".format(len(test_previousFive_pickups), len(test_previousFive_pickups[0]), len(test_previousFive_pickups)*len(test_previousFive_pickups[0])))
    #taking 70% data as train data from each cluster
    train_lat = [i[:trainsizeonestand] for i in lat]
    train_lon = [i[:trainsizeonestand] for i in lon]
    train_weekDay = [i[:trainsizeonestand] for i in day_of_week]
    train_weighted_avg = [i[:trainsizeonestand] for i in predicted_pickup_values_list]
    train_TruePickups = [i[:trainsizeonestand] for i in TruePickups]
    #taking 30% data as test data from each cluster
    test_lat = [i[trainsizeonestand:] for i in lat]
    test_lon = [i[trainsizeonestand:] for i in lon]
    test_weekDay = [i[trainsizeonestand:] for i in day_of_week]
    test_weighted_avg = [i[trainsizeonestand:] for i in predicted_pickup_values_list]
    test_TruePickups = [i[trainsizeonestand:] for i in TruePickups]
    # convert from lists of lists of list to lists of list
    train_pickups = []
    test_pickups = []
    for i in range(numberofstands):
        train_pickups.extend(train_previousFive_pickups[i])
        test_pickups.extend(test_previousFive_pickups[i])    
    train_prevPickups_freq_amp = train_pickups
    test_prevPickups_freq_amp = test_pickups
    print("Number of data points in train data = {}. Number of columns till now = {}".format(len(train_prevPickups_freq_amp), len(train_prevPickups_freq_amp[0])))
    print("Number of data points in test data = {}. Number of columns till now = {}".format(len(test_prevPickups_freq_amp), len(test_prevPickups_freq_amp[0])))
    # converting lists of lists into single list i.e flatten
    # a  = [[1,2,3,4],[4,6,7,8]]
    # print(sum(a,[]))
    # [1, 2, 3, 4, 4, 6, 7, 8]

    train_flat_lat = sum(train_lat, [])
    train_flat_lon = sum(train_lon, [])
    train_flat_weekDay = sum(train_weekDay, [])
    train_weighted_avg_flat = sum(train_weighted_avg, [])
    train_TruePickups_flat = sum(train_TruePickups, [])

    test_flat_lat = sum(test_lat, [])
    test_flat_lon = sum(test_lon, [])
    test_flat_weekDay = sum(test_weekDay, [])
    test_weighted_avg_flat = sum(test_weighted_avg, [])
    test_TruePickups_flat = sum(test_TruePickups, [])
    columns=[]
    for i in range(number_of_time_stamps):
        ftname="ft"+str(i)
        columns.append(ftname)
    #columns = ['ft_10','ft_9','ft_8','ft_7','ft_6','ft_5','ft_4','ft_3','ft_2','ft_1']
    Train_DF = pd.DataFrame(data = train_prevPickups_freq_amp, columns = columns)
    Test_DF = pd.DataFrame(data = test_prevPickups_freq_amp, columns = columns)

    trainMAPE_lr, trainMSE_lr, train_RMSE_lr, train_SMAPE_lr,testMAPE_lr, testMSE_lr, test_RMSE_lr,test_SMAPE_lr = lin_regression(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)
    trainMAPE_rf, trainMSE_rf, train_RMSE_rf, train_SMAPE_rf,testMAPE_rf, testMSE_rf, test_RMSE_rf,test_SMAPE_rf = randomFor(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)
    trainMAPE_xgb, trainMSE_xgb, train_RMSE_xgb, train_SMAPE_xgb,testMAPE_xgb, testMSE_xgb, test_RMSE_xgb,test_SMAPE_xgb = xgboost_reg(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)
    if (test_SMAPE_lr<min_sMAPE_lr):
        min_sMAPE_lr=test_SMAPE_lr
        best_number_of_time_stamps_lr = number_of_time_stamps
    if (test_SMAPE_rf<min_sMAPE_rf):
        min_sMAPE_rf=test_SMAPE_rf
        best_number_of_time_stamps_rf = number_of_time_stamps
    if (test_SMAPE_xgb<min_sMAPE_xgb):
        min_sMAPE_xgb=test_SMAPE_xgb
        best_number_of_time_stamps_xgb = number_of_time_stamps
print("The best number_of_time_stamps:  " + str(best_number_of_time_stamps_lr) + "with best sMAPE: " + str(min_sMAPE_lr))
print("The best number_of_time_stamps: " + str(best_number_of_time_stamps_rf) + "with best sMAPE: " + str(min_sMAPE_rf))
print("The best number_of_time_stamps: " + str(best_number_of_time_stamps_xgb) + "with best sMAPE: " + str(min_sMAPE_xgb))   


## Run baseline models 

In [ ]:
    # TruePickups varaible
    # it is list of lists
    # It will be used as true labels/ground truth. Now since we are taking previous 10 pickups as a training data for predicting
    # next pickup(here next pickup will be a true/ground truth pickup), so "TruePickups" will not contain first five pickups of each 
    # cluster. It will contain number of pickups 17519-10 = 17509 for each cluster. 
    TruePickups = []
    #number_of_time_stamps = 15 #Linear
    #number_of_time_stamps = 12 #Random forest
    number_of_time_stamps = 19 #XGBoost
    # lat,lon will contain 17519-10=17509 times latitude of cluster center for every stand
    # it is list of lists
    lat = []
    lon = []
    # we will code each day 
    # sunday = 0, monday=1, tue = 2, wed=3, thur=4, fri=5,sat=6
    day_of_week = []

    feat = []
    numberofstands = 63 #Number of taxi stand
    lengthoftimeseries= int(len(regionWisePickup_porto_data[0])/1)

    feat = [0]*number_of_time_stamps
    for i in range(numberofstands):
            lat.append([centerOfRegions[i][0]]*(lengthoftimeseries-number_of_time_stamps)) 
            lon.append([centerOfRegions[i][1]]*(lengthoftimeseries-number_of_time_stamps))
            day_of_week.append([int(((int(j/48)%7)+number_of_time_stamps)%7) for j in range(number_of_time_stamps, lengthoftimeseries)])
            #48 is the time bin for a day (30 min)
            feat = np.vstack((feat, [regionWisePickup_porto_data[i][k:k+number_of_time_stamps] for k in range(0, len(regionWisePickup_porto_data[i]) - (number_of_time_stamps))]))
            #feat = np.vstack(([regionWisePickup_porto_data[i][k:k+number_of_time_stamps] for k in range(0, len(regionWisePickup_porto_data[i]) - (number_of_time_stamps))]))
            TruePickups.append(regionWisePickup_porto_data[i][number_of_time_stamps:])
            #output contains pickup values of all the regions and of each time stamp, except first 5 time stamp pickups of each region.
    feat = feat[1:]
    # "predicted_pickup_values": it is a temporary array that store weighted moving avarag prediction values for each 30min intervl, 
    # for each cluster it will get reset.
    # for every cluster it contains 17519 values
    predicted_pickup_values = []

    # "predicted_pickup_values_list"
    # it is list of lists
    predicted_pickup_values_list = []

    predicted_value = -1  #it will contain cuurent predicted_value. Default is given -1 which will be replaced later

    window_size = 2
    for i in range(numberofstands):
        for j in range(lengthoftimeseries):
            if j == 0:
                predicted_value = regionWisePickup_porto_data[i][j]
                predicted_pickup_values.append(0)
            else:
                if j>=window_size:
                    sumPickups = 0
                    sumOfWeights = 0
                    for k in range(window_size, 0, -1):
                        sumPickups += k*(regionWisePickup_porto_data[i][j -window_size + (k - 1)])
                        sumOfWeights += k
                    predicted_value = int(sumPickups/sumOfWeights)
                    predicted_pickup_values.append(predicted_value)
                else:
                    sumPickups = 0
                    sumOfWeights = 0
                    for k in range(j, 0, -1):
                        sumPickups += k*regionWisePickup_porto_data[i][k-1]
                        sumOfWeights += k
                    predicted_value = int(sumPickups/sumOfWeights)
                    predicted_pickup_values.append(predicted_value)

        predicted_pickup_values_list.append(predicted_pickup_values[10:])
        predicted_pickup_values = []
    trainsize =int((lengthoftimeseries - number_of_time_stamps)*numberofstands*0.7)
    testsize=int((lengthoftimeseries - number_of_time_stamps)*numberofstands*0.3)
    trainsizeonestand=int((lengthoftimeseries - number_of_time_stamps)*0.7)
    testsizeonestand=int((lengthoftimeseries - number_of_time_stamps)*0.3)
    train_previousFive_pickups  = [feat[(i*(lengthoftimeseries - number_of_time_stamps)):((lengthoftimeseries - number_of_time_stamps)*i+trainsizeonestand)] for i in range(numberofstands)]
    test_previousFive_pickups  = [feat[(i*(lengthoftimeseries - number_of_time_stamps)+trainsizeonestand):((lengthoftimeseries - number_of_time_stamps)*(i+1))] for i in range(numberofstands)]
    print("Train Data: Total number of clusters = {}. Number of points in each cluster = {}. Total number of training points = {}".format(len(train_previousFive_pickups), len(train_previousFive_pickups[0]), len(train_previousFive_pickups)*len(train_previousFive_pickups[0])))
    print("Test Data: Total number of clusters = {}. Number of points in each cluster = {}. Total number of test points = {}".format(len(test_previousFive_pickups), len(test_previousFive_pickups[0]), len(test_previousFive_pickups)*len(test_previousFive_pickups[0])))
    #taking 70% data as train data from each cluster
    train_lat = [i[:trainsizeonestand] for i in lat]
    train_lon = [i[:trainsizeonestand] for i in lon]
    train_weekDay = [i[:trainsizeonestand] for i in day_of_week]
    train_weighted_avg = [i[:trainsizeonestand] for i in predicted_pickup_values_list]
    train_TruePickups = [i[:trainsizeonestand] for i in TruePickups]
    #taking 30% data as test data from each cluster
    test_lat = [i[trainsizeonestand:] for i in lat]
    test_lon = [i[trainsizeonestand:] for i in lon]
    test_weekDay = [i[trainsizeonestand:] for i in day_of_week]
    test_weighted_avg = [i[trainsizeonestand:] for i in predicted_pickup_values_list]
    test_TruePickups = [i[trainsizeonestand:] for i in TruePickups]
    # convert from lists of lists of list to lists of list
    train_pickups = []
    test_pickups = []
    for i in range(numberofstands):
        train_pickups.extend(train_previousFive_pickups[i])
        test_pickups.extend(test_previousFive_pickups[i])    
    train_prevPickups_freq_amp = train_pickups
    test_prevPickups_freq_amp = test_pickups
    print("Number of data points in train data = {}. Number of columns till now = {}".format(len(train_prevPickups_freq_amp), len(train_prevPickups_freq_amp[0])))
    print("Number of data points in test data = {}. Number of columns till now = {}".format(len(test_prevPickups_freq_amp), len(test_prevPickups_freq_amp[0])))
    # converting lists of lists into single list i.e flatten
    # a  = [[1,2,3,4],[4,6,7,8]]
    # print(sum(a,[]))
    # [1, 2, 3, 4, 4, 6, 7, 8]

    train_flat_lat = sum(train_lat, [])
    train_flat_lon = sum(train_lon, [])
    train_flat_weekDay = sum(train_weekDay, [])
    train_weighted_avg_flat = sum(train_weighted_avg, [])
    train_TruePickups_flat = sum(train_TruePickups, [])

    test_flat_lat = sum(test_lat, [])
    test_flat_lon = sum(test_lon, [])
    test_flat_weekDay = sum(test_weekDay, [])
    test_weighted_avg_flat = sum(test_weighted_avg, [])
    test_TruePickups_flat = sum(test_TruePickups, [])
    columns=[]
    for i in range(number_of_time_stamps):
        ftname="ft"+str(i)
        columns.append(ftname)
    #columns = ['ft_10','ft_9','ft_8','ft_7','ft_6','ft_5','ft_4','ft_3','ft_2','ft_1']
    Train_DF = pd.DataFrame(data = train_prevPickups_freq_amp, columns = columns)
    Test_DF = pd.DataFrame(data = test_prevPickups_freq_amp, columns = columns)

    #trainMAPE_lr, trainMSE_lr, train_RMSE_lr, train_SMAPE_lr,testMAPE_lr, testMSE_lr, test_RMSE_lr,test_SMAPE_lr = lin_regression(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)
    #rainMAPE_rf, trainMSE_rf, train_RMSE_rf, train_SMAPE_rf,testMAPE_rf, testMSE_rf, test_RMSE_rf,test_SMAPE_rf = randomFor(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)
    trainMAPE_xgb, trainMSE_xgb, train_RMSE_xgb, train_SMAPE_xgb,testMAPE_xgb, testMSE_xgb, test_RMSE_xgb,test_SMAPE_xgb = xgboost_reg(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)    


In [ ]:
error_table_regressions = pd.DataFrame(columns = ["Model", "TrainMAPE(%)", "TrainMSE", "TrainRMSE","TrainSMAPE(%)","TestMAPE(%)", "TestMSE", "TestRMSE","TestSMAPE(%)"])
error_table_regressions = error_table_regressions.append(pd.DataFrame([["Linear Regression", trainMAPE_lr*100, trainMSE_lr, train_RMSE_lr, train_SMAPE_lr, testMAPE_lr*100, testMSE_lr,test_RMSE_lr,test_SMAPE_lr ]], columns = ["Model", "TrainMAPE(%)", "TrainMSE", "TrainRMSE","TrainSMAPE(%)","TestMAPE(%)", "TestMSE","TestRMSE","TestSMAPE(%)"]))

In [ ]:
error_table_regressions = error_table_regressions.append(pd.DataFrame([["Random Forest Regression", trainMAPE_rf*100, trainMSE_rf, train_RMSE_rf, train_SMAPE_rf, testMAPE_rf*100, testMSE_rf, test_RMSE_rf, test_SMAPE_rf]], columns = ["Model", "TrainMAPE(%)", "TrainMSE","TrainRMSE","TrainSMAPE(%)", "TestMAPE(%)", "TestMSE","TestRMSE","TestSMAPE(%)"]))

In [ ]:
error_table_regressions = error_table_regressions.append(pd.DataFrame([["XGBoost Regressor", trainMAPE_xgb*100, trainMSE_xgb,train_RMSE_xgb, train_SMAPE_xgb, testMAPE_xgb*100, testMSE_xgb,test_RMSE_xgb, test_SMAPE_xgb]], columns = ["Model", "TrainMAPE(%)", "TrainMSE", "TrainRMSE","TrainSMAPE(%)","TestMAPE(%)", "TestMSE","TestRMSE","TestSMAPE(%)"]))
error_table_regressions.reset_index(drop = True, inplace = True)

In [ ]:
error_table_regressions

## Linear Regression

In [ ]:
numberoftaxistands = 63
atrainMSE_lr = []
atestMSE_lr = []
atrain_SMAPE_lr =[]
atest_SMAPE_lr = []
aalpha_lr=[]
for ii in range(numberoftaxistands):
    print("Stand: " + str(ii+1))
    # TruePickups varaible
    # it is list of lists
    # It will be used as true labels/ground truth. Now since we are taking previous 10 pickups as a training data for predicting
    # next pickup(here next pickup will be a true/ground truth pickup), so "TruePickups" will not contain first five pickups of each 
    # cluster. It will contain number of pickups 17519-10 = 17509 for each cluster. 
    TruePickups = []
    number_of_time_stamps = 15 #Linear
    #number_of_time_stamps = 12 #Random forest
    #number_of_time_stamps = 19 #XGBoost
    # lat,lon will contain 17519-10=17509 times latitude of cluster center for every stand
    # it is list of lists
    lat = []
    lon = []
    # we will code each day 
    # sunday = 0, monday=1, tue = 2, wed=3, thur=4, fri=5,sat=6
    day_of_week = []

    feat = []
    numberofstands = 1 #Number of taxi stand
    lengthoftimeseries= int(len(regionWisePickup_porto_data[0])/1)

    feat = [0]*number_of_time_stamps
    for i in range(numberofstands):
            lat.append([centerOfRegions[ii][0]]*(lengthoftimeseries-number_of_time_stamps)) 
            lon.append([centerOfRegions[ii][1]]*(lengthoftimeseries-number_of_time_stamps))
            day_of_week.append([int(((int(j/48)%7)+number_of_time_stamps)%7) for j in range(number_of_time_stamps, lengthoftimeseries)])
            #48 is the time bin for a day (30 min)
            feat = np.vstack((feat, [regionWisePickup_porto_data[ii][k:k+number_of_time_stamps] for k in range(0, len(regionWisePickup_porto_data[ii]) - (number_of_time_stamps))]))
            #feat = np.vstack(([regionWisePickup_porto_data[i][k:k+number_of_time_stamps] for k in range(0, len(regionWisePickup_porto_data[i]) - (number_of_time_stamps))]))
            TruePickups.append(regionWisePickup_porto_data[ii][number_of_time_stamps:])
            #output contains pickup values of all the regions and of each time stamp, except first 5 time stamp pickups of each region.
    feat = feat[1:]
    # "predicted_pickup_values": it is a temporary array that store weighted moving avarag prediction values for each 30min intervl, 
    # for each cluster it will get reset.
    # for every cluster it contains 17519 values
    predicted_pickup_values = []

    # "predicted_pickup_values_list"
    # it is list of lists
    predicted_pickup_values_list = []

    predicted_value = -1  #it will contain cuurent predicted_value. Default is given -1 which will be replaced later

    window_size = 2
    for i in range(numberofstands):
        for j in range(lengthoftimeseries):
            if j == 0:
                predicted_value = regionWisePickup_porto_data[ii][j]
                predicted_pickup_values.append(0)
            else:
                if j>=window_size:
                    sumPickups = 0
                    sumOfWeights = 0
                    for k in range(window_size, 0, -1):
                        sumPickups += k*(regionWisePickup_porto_data[ii][j -window_size + (k - 1)])
                        sumOfWeights += k
                    predicted_value = int(sumPickups/sumOfWeights)
                    predicted_pickup_values.append(predicted_value)
                else:
                    sumPickups = 0
                    sumOfWeights = 0
                    for k in range(j, 0, -1):
                        sumPickups += k*regionWisePickup_porto_data[ii][k-1]
                        sumOfWeights += k
                    predicted_value = int(sumPickups/sumOfWeights)
                    predicted_pickup_values.append(predicted_value)

        predicted_pickup_values_list.append(predicted_pickup_values[10:])
        predicted_pickup_values = []
    trainsize =int((lengthoftimeseries - number_of_time_stamps)*numberofstands*0.7)
    testsize=int((lengthoftimeseries - number_of_time_stamps)*numberofstands*0.3)
    trainsizeonestand=int((lengthoftimeseries - number_of_time_stamps)*0.7)
    testsizeonestand=int((lengthoftimeseries - number_of_time_stamps)*0.3)
    train_previousFive_pickups  = [feat[(i*(lengthoftimeseries - number_of_time_stamps)):((lengthoftimeseries - number_of_time_stamps)*i+trainsizeonestand)] for i in range(numberofstands)]
    test_previousFive_pickups  = [feat[(i*(lengthoftimeseries - number_of_time_stamps)+trainsizeonestand):((lengthoftimeseries - number_of_time_stamps)*(i+1))] for i in range(numberofstands)]
    print("Train Data: Total number of clusters = {}. Number of points in each cluster = {}. Total number of training points = {}".format(len(train_previousFive_pickups), len(train_previousFive_pickups[0]), len(train_previousFive_pickups)*len(train_previousFive_pickups[0])))
    print("Test Data: Total number of clusters = {}. Number of points in each cluster = {}. Total number of test points = {}".format(len(test_previousFive_pickups), len(test_previousFive_pickups[0]), len(test_previousFive_pickups)*len(test_previousFive_pickups[0])))
    #taking 70% data as train data from each cluster
    train_lat = [i[:trainsizeonestand] for i in lat]
    train_lon = [i[:trainsizeonestand] for i in lon]
    train_weekDay = [i[:trainsizeonestand] for i in day_of_week]
    train_weighted_avg = [i[:trainsizeonestand] for i in predicted_pickup_values_list]
    train_TruePickups = [i[:trainsizeonestand] for i in TruePickups]
    #taking 30% data as test data from each cluster
    test_lat = [i[trainsizeonestand:] for i in lat]
    test_lon = [i[trainsizeonestand:] for i in lon]
    test_weekDay = [i[trainsizeonestand:] for i in day_of_week]
    test_weighted_avg = [i[trainsizeonestand:] for i in predicted_pickup_values_list]
    test_TruePickups = [i[trainsizeonestand:] for i in TruePickups]
    # convert from lists of lists of list to lists of list
    train_pickups = []
    test_pickups = []
    for i in range(numberofstands):
        train_pickups.extend(train_previousFive_pickups[i])
        test_pickups.extend(test_previousFive_pickups[i])    
    train_prevPickups_freq_amp = train_pickups
    test_prevPickups_freq_amp = test_pickups
    print("Number of data points in train data = {}. Number of columns till now = {}".format(len(train_prevPickups_freq_amp), len(train_prevPickups_freq_amp[0])))
    print("Number of data points in test data = {}. Number of columns till now = {}".format(len(test_prevPickups_freq_amp), len(test_prevPickups_freq_amp[0])))
    # converting lists of lists into single list i.e flatten
    # a  = [[1,2,3,4],[4,6,7,8]]
    # print(sum(a,[]))
    # [1, 2, 3, 4, 4, 6, 7, 8]

    train_flat_lat = sum(train_lat, [])
    train_flat_lon = sum(train_lon, [])
    train_flat_weekDay = sum(train_weekDay, [])
    train_weighted_avg_flat = sum(train_weighted_avg, [])
    train_TruePickups_flat = sum(train_TruePickups, [])

    test_flat_lat = sum(test_lat, [])
    test_flat_lon = sum(test_lon, [])
    test_flat_weekDay = sum(test_weekDay, [])
    test_weighted_avg_flat = sum(test_weighted_avg, [])
    test_TruePickups_flat = sum(test_TruePickups, [])
    columns=[]
    for i in range(number_of_time_stamps):
        ftname="ft"+str(i)
        columns.append(ftname)
    #columns = ['ft_10','ft_9','ft_8','ft_7','ft_6','ft_5','ft_4','ft_3','ft_2','ft_1']
    Train_DF = pd.DataFrame(data = train_prevPickups_freq_amp, columns = columns)
    Test_DF = pd.DataFrame(data = test_prevPickups_freq_amp, columns = columns)

    trainMAPE_lr, trainMSE_lr, train_RMSE_lr, train_SMAPE_lr,testMAPE_lr, testMSE_lr, test_RMSE_lr,test_SMAPE_lr,alpha_lr = lin_regression(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)
    #rainMAPE_rf, trainMSE_rf, train_RMSE_rf, train_SMAPE_rf,testMAPE_rf, testMSE_rf, test_RMSE_rf,test_SMAPE_rf = randomFor(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)
    #trainMAPE_xgb, trainMSE_xgb, train_RMSE_xgb, train_SMAPE_xgb,testMAPE_xgb, testMSE_xgb, test_RMSE_xgb,test_SMAPE_xgb = xgboost_reg(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)    

    aalpha_lr.append(alpha_lr)
    atrainMSE_lr.append(trainMSE_lr)
    atestMSE_lr.append(testMSE_lr)
    atrain_SMAPE_lr.append(train_SMAPE_lr)
    atest_SMAPE_lr.append(test_SMAPE_lr)
    
print("Train MSE:")
for element in atrainMSE_lr:
    print(element)
print("Test MSE:")
for element in atestMSE_lr:
    print(element)
print("Train sMAPE:")
for element in atrain_SMAPE_lr:
    print(element)
print("Test sMAPE:")
for element in atest_SMAPE_lr:
    print(element)
print("Alpha:")
for element in aalpha_lr:
    print(element)    

## Random forest

In [ ]:
numberoftaxistands = 63
atrainMSE_rf = []
atestMSE_rf= []
atrain_SMAPE_rf =[]
atest_SMAPE_rf = []
atree_rf=[]
for ii in range(numberoftaxistands):
    print("Stand: " + str(ii+1))
    # TruePickups varaible
    # it is list of lists
    # It will be used as true labels/ground truth. Now since we are taking previous 10 pickups as a training data for predicting
    # next pickup(here next pickup will be a true/ground truth pickup), so "TruePickups" will not contain first five pickups of each 
    # cluster. It will contain number of pickups 17519-10 = 17509 for each cluster. 
    TruePickups = []
    #number_of_time_stamps = 15 #Linear
    number_of_time_stamps = 12 #Random forest
    #number_of_time_stamps = 19 #XGBoost
    # lat,lon will contain 17519-10=17509 times latitude of cluster center for every stand
    # it is list of lists
    lat = []
    lon = []
    # we will code each day 
    # sunday = 0, monday=1, tue = 2, wed=3, thur=4, fri=5,sat=6
    day_of_week = []

    feat = []
    numberofstands = 1 #Number of taxi stand
    lengthoftimeseries= int(len(regionWisePickup_porto_data[0])/1)

    feat = [0]*number_of_time_stamps
    for i in range(numberofstands):
            lat.append([centerOfRegions[ii][0]]*(lengthoftimeseries-number_of_time_stamps)) 
            lon.append([centerOfRegions[ii][1]]*(lengthoftimeseries-number_of_time_stamps))
            day_of_week.append([int(((int(j/48)%7)+number_of_time_stamps)%7) for j in range(number_of_time_stamps, lengthoftimeseries)])
            #48 is the time bin for a day (30 min)
            feat = np.vstack((feat, [regionWisePickup_porto_data[ii][k:k+number_of_time_stamps] for k in range(0, len(regionWisePickup_porto_data[ii]) - (number_of_time_stamps))]))
            #feat = np.vstack(([regionWisePickup_porto_data[i][k:k+number_of_time_stamps] for k in range(0, len(regionWisePickup_porto_data[i]) - (number_of_time_stamps))]))
            TruePickups.append(regionWisePickup_porto_data[ii][number_of_time_stamps:])
            #output contains pickup values of all the regions and of each time stamp, except first 5 time stamp pickups of each region.
    feat = feat[1:]
    # "predicted_pickup_values": it is a temporary array that store weighted moving avarag prediction values for each 30min intervl, 
    # for each cluster it will get reset.
    # for every cluster it contains 17519 values
    predicted_pickup_values = []

    # "predicted_pickup_values_list"
    # it is list of lists
    predicted_pickup_values_list = []

    predicted_value = -1  #it will contain cuurent predicted_value. Default is given -1 which will be replaced later

    window_size = 2
    for i in range(numberofstands):
        for j in range(lengthoftimeseries):
            if j == 0:
                predicted_value = regionWisePickup_porto_data[ii][j]
                predicted_pickup_values.append(0)
            else:
                if j>=window_size:
                    sumPickups = 0
                    sumOfWeights = 0
                    for k in range(window_size, 0, -1):
                        sumPickups += k*(regionWisePickup_porto_data[ii][j -window_size + (k - 1)])
                        sumOfWeights += k
                    predicted_value = int(sumPickups/sumOfWeights)
                    predicted_pickup_values.append(predicted_value)
                else:
                    sumPickups = 0
                    sumOfWeights = 0
                    for k in range(j, 0, -1):
                        sumPickups += k*regionWisePickup_porto_data[ii][k-1]
                        sumOfWeights += k
                    predicted_value = int(sumPickups/sumOfWeights)
                    predicted_pickup_values.append(predicted_value)

        predicted_pickup_values_list.append(predicted_pickup_values[10:])
        predicted_pickup_values = []
    trainsize =int((lengthoftimeseries - number_of_time_stamps)*numberofstands*0.7)
    testsize=int((lengthoftimeseries - number_of_time_stamps)*numberofstands*0.3)
    trainsizeonestand=int((lengthoftimeseries - number_of_time_stamps)*0.7)
    testsizeonestand=int((lengthoftimeseries - number_of_time_stamps)*0.3)
    train_previousFive_pickups  = [feat[(i*(lengthoftimeseries - number_of_time_stamps)):((lengthoftimeseries - number_of_time_stamps)*i+trainsizeonestand)] for i in range(numberofstands)]
    test_previousFive_pickups  = [feat[(i*(lengthoftimeseries - number_of_time_stamps)+trainsizeonestand):((lengthoftimeseries - number_of_time_stamps)*(i+1))] for i in range(numberofstands)]
    print("Train Data: Total number of clusters = {}. Number of points in each cluster = {}. Total number of training points = {}".format(len(train_previousFive_pickups), len(train_previousFive_pickups[0]), len(train_previousFive_pickups)*len(train_previousFive_pickups[0])))
    print("Test Data: Total number of clusters = {}. Number of points in each cluster = {}. Total number of test points = {}".format(len(test_previousFive_pickups), len(test_previousFive_pickups[0]), len(test_previousFive_pickups)*len(test_previousFive_pickups[0])))
    #taking 70% data as train data from each cluster
    train_lat = [i[:trainsizeonestand] for i in lat]
    train_lon = [i[:trainsizeonestand] for i in lon]
    train_weekDay = [i[:trainsizeonestand] for i in day_of_week]
    train_weighted_avg = [i[:trainsizeonestand] for i in predicted_pickup_values_list]
    train_TruePickups = [i[:trainsizeonestand] for i in TruePickups]
    #taking 30% data as test data from each cluster
    test_lat = [i[trainsizeonestand:] for i in lat]
    test_lon = [i[trainsizeonestand:] for i in lon]
    test_weekDay = [i[trainsizeonestand:] for i in day_of_week]
    test_weighted_avg = [i[trainsizeonestand:] for i in predicted_pickup_values_list]
    test_TruePickups = [i[trainsizeonestand:] for i in TruePickups]
    # convert from lists of lists of list to lists of list
    train_pickups = []
    test_pickups = []
    for i in range(numberofstands):
        train_pickups.extend(train_previousFive_pickups[i])
        test_pickups.extend(test_previousFive_pickups[i])    
    train_prevPickups_freq_amp = train_pickups
    test_prevPickups_freq_amp = test_pickups
    print("Number of data points in train data = {}. Number of columns till now = {}".format(len(train_prevPickups_freq_amp), len(train_prevPickups_freq_amp[0])))
    print("Number of data points in test data = {}. Number of columns till now = {}".format(len(test_prevPickups_freq_amp), len(test_prevPickups_freq_amp[0])))
    # converting lists of lists into single list i.e flatten
    # a  = [[1,2,3,4],[4,6,7,8]]
    # print(sum(a,[]))
    # [1, 2, 3, 4, 4, 6, 7, 8]

    train_flat_lat = sum(train_lat, [])
    train_flat_lon = sum(train_lon, [])
    train_flat_weekDay = sum(train_weekDay, [])
    train_weighted_avg_flat = sum(train_weighted_avg, [])
    train_TruePickups_flat = sum(train_TruePickups, [])

    test_flat_lat = sum(test_lat, [])
    test_flat_lon = sum(test_lon, [])
    test_flat_weekDay = sum(test_weekDay, [])
    test_weighted_avg_flat = sum(test_weighted_avg, [])
    test_TruePickups_flat = sum(test_TruePickups, [])
    columns=[]
    for i in range(number_of_time_stamps):
        ftname="ft"+str(i)
        columns.append(ftname)
    #columns = ['ft_10','ft_9','ft_8','ft_7','ft_6','ft_5','ft_4','ft_3','ft_2','ft_1']
    Train_DF = pd.DataFrame(data = train_prevPickups_freq_amp, columns = columns)
    Test_DF = pd.DataFrame(data = test_prevPickups_freq_amp, columns = columns)

    #trainMAPE_lr, trainMSE_lr, train_RMSE_lr, train_SMAPE_lr,testMAPE_lr, testMSE_lr, test_RMSE_lr,test_SMAPE_lr = lin_regression(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)
    rainMAPE_rf, trainMSE_rf, train_RMSE_rf, train_SMAPE_rf,testMAPE_rf, testMSE_rf, test_RMSE_rf,test_SMAPE_rf, params = randomFor(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)
    #trainMAPE_xgb, trainMSE_xgb, train_RMSE_xgb, train_SMAPE_xgb,testMAPE_xgb, testMSE_xgb, test_RMSE_xgb,test_SMAPE_xgb = xgboost_reg(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)    

    atree_rf.append(params)
    atrainMSE_rf.append(trainMSE_rf)
    atestMSE_rf.append(testMSE_rf)
    atrain_SMAPE_rf.append(train_SMAPE_rf)
    atest_SMAPE_rf.append(test_SMAPE_rf)
    
print("Train MSE:")
for element in atrainMSE_rf:
    print(element)
print("Test MSE:")
for element in atestMSE_rf:
    print(element)
print("Train sMAPE:")
for element in atrain_SMAPE_rf:
    print(element)
print("Test sMAPE:")
for element in atest_SMAPE_rf:
    print(element)
print("Number of tree:")
for element in atree_rf:
    print(element)

## XGBoost 

In [ ]:
numberoftaxistands = 63
atrainMSE_xgb = []
atestMSE_xgb= []
atrain_SMAPE_xgb =[]
atest_SMAPE_xgb = []
adepth_xgb=[]
atree_xgb=[]
for ii in range(numberoftaxistands):
    print("Stand: " + str(ii+1))
    # TruePickups varaible
    # it is list of lists
    # It will be used as true labels/ground truth. Now since we are taking previous 10 pickups as a training data for predicting
    # next pickup(here next pickup will be a true/ground truth pickup), so "TruePickups" will not contain first five pickups of each 
    # cluster. It will contain number of pickups 17519-10 = 17509 for each cluster. 
    TruePickups = []
    #number_of_time_stamps = 15 #Linear
    #number_of_time_stamps = 12 #Random forest
    number_of_time_stamps = 19 #XGBoost
    # lat,lon will contain 17519-10=17509 times latitude of cluster center for every stand
    # it is list of lists
    lat = []
    lon = []
    # we will code each day 
    # sunday = 0, monday=1, tue = 2, wed=3, thur=4, fri=5,sat=6
    day_of_week = []

    feat = []
    numberofstands = 1 #Number of taxi stand
    lengthoftimeseries= int(len(regionWisePickup_porto_data[0])/1)

    feat = [0]*number_of_time_stamps
    for i in range(numberofstands):
            lat.append([centerOfRegions[ii][0]]*(lengthoftimeseries-number_of_time_stamps)) 
            lon.append([centerOfRegions[ii][1]]*(lengthoftimeseries-number_of_time_stamps))
            day_of_week.append([int(((int(j/48)%7)+number_of_time_stamps)%7) for j in range(number_of_time_stamps, lengthoftimeseries)])
            #48 is the time bin for a day (30 min)
            feat = np.vstack((feat, [regionWisePickup_porto_data[ii][k:k+number_of_time_stamps] for k in range(0, len(regionWisePickup_porto_data[ii]) - (number_of_time_stamps))]))
            #feat = np.vstack(([regionWisePickup_porto_data[i][k:k+number_of_time_stamps] for k in range(0, len(regionWisePickup_porto_data[i]) - (number_of_time_stamps))]))
            TruePickups.append(regionWisePickup_porto_data[ii][number_of_time_stamps:])
            #output contains pickup values of all the regions and of each time stamp, except first 5 time stamp pickups of each region.
    feat = feat[1:]
    # "predicted_pickup_values": it is a temporary array that store weighted moving avarag prediction values for each 30min intervl, 
    # for each cluster it will get reset.
    # for every cluster it contains 17519 values
    predicted_pickup_values = []

    # "predicted_pickup_values_list"
    # it is list of lists
    predicted_pickup_values_list = []

    predicted_value = -1  #it will contain cuurent predicted_value. Default is given -1 which will be replaced later

    window_size = 2
    for i in range(numberofstands):
        for j in range(lengthoftimeseries):
            if j == 0:
                predicted_value = regionWisePickup_porto_data[ii][j]
                predicted_pickup_values.append(0)
            else:
                if j>=window_size:
                    sumPickups = 0
                    sumOfWeights = 0
                    for k in range(window_size, 0, -1):
                        sumPickups += k*(regionWisePickup_porto_data[ii][j -window_size + (k - 1)])
                        sumOfWeights += k
                    predicted_value = int(sumPickups/sumOfWeights)
                    predicted_pickup_values.append(predicted_value)
                else:
                    sumPickups = 0
                    sumOfWeights = 0
                    for k in range(j, 0, -1):
                        sumPickups += k*regionWisePickup_porto_data[ii][k-1]
                        sumOfWeights += k
                    predicted_value = int(sumPickups/sumOfWeights)
                    predicted_pickup_values.append(predicted_value)

        predicted_pickup_values_list.append(predicted_pickup_values[10:])
        predicted_pickup_values = []
    trainsize =int((lengthoftimeseries - number_of_time_stamps)*numberofstands*0.7)
    testsize=int((lengthoftimeseries - number_of_time_stamps)*numberofstands*0.3)
    trainsizeonestand=int((lengthoftimeseries - number_of_time_stamps)*0.7)
    testsizeonestand=int((lengthoftimeseries - number_of_time_stamps)*0.3)
    train_previousFive_pickups  = [feat[(i*(lengthoftimeseries - number_of_time_stamps)):((lengthoftimeseries - number_of_time_stamps)*i+trainsizeonestand)] for i in range(numberofstands)]
    test_previousFive_pickups  = [feat[(i*(lengthoftimeseries - number_of_time_stamps)+trainsizeonestand):((lengthoftimeseries - number_of_time_stamps)*(i+1))] for i in range(numberofstands)]
    print("Train Data: Total number of clusters = {}. Number of points in each cluster = {}. Total number of training points = {}".format(len(train_previousFive_pickups), len(train_previousFive_pickups[0]), len(train_previousFive_pickups)*len(train_previousFive_pickups[0])))
    print("Test Data: Total number of clusters = {}. Number of points in each cluster = {}. Total number of test points = {}".format(len(test_previousFive_pickups), len(test_previousFive_pickups[0]), len(test_previousFive_pickups)*len(test_previousFive_pickups[0])))
    #taking 70% data as train data from each cluster
    train_lat = [i[:trainsizeonestand] for i in lat]
    train_lon = [i[:trainsizeonestand] for i in lon]
    train_weekDay = [i[:trainsizeonestand] for i in day_of_week]
    train_weighted_avg = [i[:trainsizeonestand] for i in predicted_pickup_values_list]
    train_TruePickups = [i[:trainsizeonestand] for i in TruePickups]
    #taking 30% data as test data from each cluster
    test_lat = [i[trainsizeonestand:] for i in lat]
    test_lon = [i[trainsizeonestand:] for i in lon]
    test_weekDay = [i[trainsizeonestand:] for i in day_of_week]
    test_weighted_avg = [i[trainsizeonestand:] for i in predicted_pickup_values_list]
    test_TruePickups = [i[trainsizeonestand:] for i in TruePickups]
    # convert from lists of lists of list to lists of list
    train_pickups = []
    test_pickups = []
    for i in range(numberofstands):
        train_pickups.extend(train_previousFive_pickups[i])
        test_pickups.extend(test_previousFive_pickups[i])    
    train_prevPickups_freq_amp = train_pickups
    test_prevPickups_freq_amp = test_pickups
    print("Number of data points in train data = {}. Number of columns till now = {}".format(len(train_prevPickups_freq_amp), len(train_prevPickups_freq_amp[0])))
    print("Number of data points in test data = {}. Number of columns till now = {}".format(len(test_prevPickups_freq_amp), len(test_prevPickups_freq_amp[0])))
    # converting lists of lists into single list i.e flatten
    # a  = [[1,2,3,4],[4,6,7,8]]
    # print(sum(a,[]))
    # [1, 2, 3, 4, 4, 6, 7, 8]

    train_flat_lat = sum(train_lat, [])
    train_flat_lon = sum(train_lon, [])
    train_flat_weekDay = sum(train_weekDay, [])
    train_weighted_avg_flat = sum(train_weighted_avg, [])
    train_TruePickups_flat = sum(train_TruePickups, [])

    test_flat_lat = sum(test_lat, [])
    test_flat_lon = sum(test_lon, [])
    test_flat_weekDay = sum(test_weekDay, [])
    test_weighted_avg_flat = sum(test_weighted_avg, [])
    test_TruePickups_flat = sum(test_TruePickups, [])
    columns=[]
    for i in range(number_of_time_stamps):
        ftname="ft"+str(i)
        columns.append(ftname)
    #columns = ['ft_10','ft_9','ft_8','ft_7','ft_6','ft_5','ft_4','ft_3','ft_2','ft_1']
    Train_DF = pd.DataFrame(data = train_prevPickups_freq_amp, columns = columns)
    Test_DF = pd.DataFrame(data = test_prevPickups_freq_amp, columns = columns)

    #trainMAPE_lr, trainMSE_lr, train_RMSE_lr, train_SMAPE_lr,testMAPE_lr, testMSE_lr, test_RMSE_lr,test_SMAPE_lr = lin_regression(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)
    #rainMAPE_rf, trainMSE_rf, train_RMSE_rf, train_SMAPE_rf,testMAPE_rf, testMSE_rf, test_RMSE_rf,test_SMAPE_rf = randomFor(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)
    trainMAPE_xgb, trainMSE_xgb, train_RMSE_xgb, train_SMAPE_xgb,testMAPE_xgb, testMSE_xgb, test_RMSE_xgb,test_SMAPE_xgb,depth_xgb,tree_xgb = xgboost_reg(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)    

    adepth_xgb.append(depth_xgb)
    atree_xgb.append(tree_xgb)
    atrainMSE_xgb.append(trainMSE_xgb)
    atestMSE_xgb.append(testMSE_xgb)
    atrain_SMAPE_xgb.append(train_SMAPE_xgb)
    atest_SMAPE_xgb.append(test_SMAPE_xgb)
    
print("Train MSE:")
for element in atrainMSE_xgb:
    print(element)
print("Test MSE:")
for element in atestMSE_xgb:
    print(element)
print("Train sMAPE:")
for element in atrain_SMAPE_xgb:
    print(element)
print("Test sMAPE:")
for element in atest_SMAPE_xgb:
    print(element)
print("Depth:")
for element in adepth_xgb:
    print(element)
print("Tree:")
for element in atree_xgb:
    print(element)

# Simple and traditional models

## Moving Average Prediction

In [ ]:
def movingaverage(values, window):
    weights = np.repeat(1.0, window)/window
    predict = np.convolve(values, weights,'valid')
    s_mape = smape(values[window-1:],predict)
    mse = mean_squared_error(values[window-1:],predict)
    return mse,s_mape

In [ ]:
def moving_average_prediction(dataframe,numberofcluster,window):
    mse=[]
    s_mape=[]        
    for i in range(numberofcluster):
        #if (i in count_points_new): ## after adjusted
            dt=dataframe[i]
            df = pd.DataFrame({'pickup':dt})
            dataset=df.values        
            m,s = movingaverage(dataset[:,0], window)
            mse.append(m)
            s_mape.append(s)
    mean_mse = np.mean(mse)
    mean_smape = np.mean(s_mape)
    return mse,s_mape        

In [ ]:
MSE_ma,SMAPE_ma = moving_average_prediction(regionWisePickup_porto_data,63,20)

In [ ]:
#Define SMAPE
#{\displaystyle {\text{SMAPE}}={\frac {100\%}{n}}\sum _{t=1}^{n}{\frac {|F_{t}-A_{t}|}{|A_{t}|+|F_{t}|+c}}}
#in our case c=1, use laplace correction
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)+1) / 100.0
    diff = np.abs(y_true - y_pred) / denominator
    #diff[denominator == 0] = 1
    return np.nanmean(diff)

# LSTM models

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)


# fix random seed for reproducibility
np.random.seed(7)
#Memory
import psutil

In [ ]:
import operator
from pandas import concat
from numpy import concatenate

#Find the closet neighbour
def getNeighbors(k):
    k_neighbour=[]
    for i in range(63):        
        k_neighbour.append([i])
    #find    
    for i in range(63):
        #if (i in count_points_new):
            distances = []
            for j in range(63):
                if (i!=j):#and (j in count_points_new):
                    dist=gpxpy.geo.haversine_distance(centroids[i][0],centroids[i][1],centroids[j][0],centroids[j][1])
                    distances.append((j, dist))
                    distances.sort(key=operator.itemgetter(1))
            for j in range(k):
                k_neighbour[i].append(distances[j][0])
        
    return k_neighbour

In [ ]:
k_neighbour = getNeighbors(62)

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

## LSTM turning parameter

### 1. Turning look_back value 

In [ ]:
best_look_back=2
min_sMAPE_lstm = 100

k=0
for look_back in range(2,20):    
        
        #Dataset for LSTM
        dt=np.array([[j] for j in regionWisePickup_porto_data[0]])
        #Take neighbour
        #for h in range(k):
        #    dt1=np.array([[j] for j in regionWisePickup_porto_data[k_neighbour[i][h+1]]])
        #    dt=np.concatenate((dt,dt1),axis=1)
        #Generate a time series with count of POI for stand
        #standpoi=np.full((len(regionWisePickup_porto_data[i]), 1), poi[i])
        #dt=np.concatenate((dt,standpoi),axis=1)
        # ensure all data is float
        values = dt.astype('float32')
        # normalize features
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(values)
        #look_back = 10 #number of data look back - to predict, default 1
        # frame as supervised learning
        reframed = series_to_supervised(scaled, look_back, 1)
        # split into train and test sets
        values = reframed.values
        train_size = int(len(values) * 0.7)
        test_size = len(values) - train_size
        #n_train_hours
        train = values[:train_size, :]
        test = values[train_size:, :]
        # split into input and outputs
        n_features = k + 1 # include itself and one extended feature
        #n_features = k + 1
        n_obs = look_back * n_features
        train_X, train_y = train[:, :n_obs], train[:, -n_features]
        test_X, test_y = test[:, :n_obs], test[:, -n_features]
        #print(train_X.shape, len(train_X), train_y.shape)
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], look_back, n_features))
        test_X = test_X.reshape((test_X.shape[0], look_back, n_features))
        #print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
        # design network
        model = Sequential()
        model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
        #model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2]),return_sequences=True))
        #model.add(LSTM(100,return_sequences=True))
        #model.add(LSTM(50,return_sequences=True))
        #model.add(LSTM(100))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='adam')
        model.summary()
        # fit network
        history = model.fit(train_X, train_y, epochs=10, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # plot history
        #plt.plot(history.history['loss'], label='train')
        #plt.plot(history.history['val_loss'], label='test')
        #plt.legend()
        #plt.show()
        #figname="Plot"+str(i)+".png"
        #plt.savefig(figname,formatt="png")
        
        # make a prediction of train set
        ytrainpredict = model.predict(train_X)
        train_X = train_X.reshape((train_X.shape[0], look_back*n_features))
        inv_ytrainpredict = concatenate((ytrainpredict, train_X[:, -(n_features-1):]), axis=1)
        inv_ytrainpredict = scaler.inverse_transform(inv_ytrainpredict)
        inv_ytrainpredict = inv_ytrainpredict[:,0]
        train_y = train_y.reshape((len(train_y), 1))
        inv_trainy = concatenate((train_y, train_X[:, -(n_features-1):]), axis=1)
        inv_trainy = scaler.inverse_transform(inv_trainy)
        inv_trainy = inv_trainy[:,0]
        #calculate score
        trainMSE = mean_squared_error(inv_trainy, inv_ytrainpredict)
        
        trainRMSE = math.sqrt(trainMSE)
        
        train_MAPE = mean_absolute_error(inv_trainy, inv_ytrainpredict)/ (sum(inv_trainy)/len(inv_trainy)) * 100
        
        train_SMAPE = smape(inv_trainy, inv_ytrainpredict)
        
        #print('Train MSE: %.3f' %trainMSE)
        #print('Train RMSE: %.3f' %trainRMSE)
        #print('Train MAPE: %.3f' %train_MAPE)
        #print('Train SMAPE: %.3f' %train_SMAPE)
        
        
        # make a prediction of test set
        ytestpredtict = model.predict(test_X)        
        test_X = test_X.reshape((test_X.shape[0], look_back*n_features))
        # invert scaling for forecast
        inv_ytestpredict = concatenate((ytestpredtict, test_X[:, -(n_features-1):]), axis=1)
        inv_ytestpredict = scaler.inverse_transform(inv_ytestpredict)
        inv_ytestpredict = inv_ytestpredict[:,0]
        # invert scaling for actual
        test_y = test_y.reshape((len(test_y), 1))
        inv_y = concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:,0]
        # calculate score
        testMSE = mean_squared_error(inv_y, inv_ytestpredict)
        
        testRMSE = math.sqrt(testMSE)
        
        test_MAPE = mean_absolute_error(inv_y, inv_ytestpredict)/ (sum(inv_y)/len(inv_y)) * 100
        
        test_SMAPE = smape(inv_y, inv_ytestpredict)
        
        #rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
        #print('Test MSE: %.3f' % testMSE)
        #print('Test RMSE: %.3f' % testRMSE)  
        #print('Test MAPE: %.3f' % test_MAPE)
        print('Look back value: ' + str(look_back))    
        print('Test SMAPE: %.3f' % test_SMAPE)   
        if (test_SMAPE<min_sMAPE_lstm):
            min_sMAPE_lstm=test_SMAPE
            best_look_back=look_back
print('The best look back value: ' + str(best_look_back)+ ", sMAPE: " + str(min_sMAPE_lstm))    

### 2. Optimizer Algorithm

In [ ]:
optimizer_al = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
best_optimizer=0
min_sMAPE_lstm = 100

k=0
for opt in range(len(optimizer_al)):    
        
        #Dataset for LSTM
        dt=np.array([[j] for j in regionWisePickup_porto_data[0]])
        #Take neighbour
        #for h in range(k):
        #    dt1=np.array([[j] for j in regionWisePickup_porto_data[k_neighbour[i][h+1]]])
        #    dt=np.concatenate((dt,dt1),axis=1)
        #Generate a time series with count of POI for stand
        #standpoi=np.full((len(regionWisePickup_porto_data[i]), 1), poi[i])
        #dt=np.concatenate((dt,standpoi),axis=1)
        # ensure all data is float
        values = dt.astype('float32')
        # normalize features
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(values)
        #look_back = 10 #number of data look back - to predict, default 1
        # frame as supervised learning
        reframed = series_to_supervised(scaled, look_back, 1)
        # split into train and test sets
        values = reframed.values
        train_size = int(len(values) * 0.7)
        test_size = len(values) - train_size
        #n_train_hours
        train = values[:train_size, :]
        test = values[train_size:, :]
        # split into input and outputs
        n_features = k + 1 # include itself and one extended feature
        #n_features = k + 1
        n_obs = look_back * n_features
        train_X, train_y = train[:, :n_obs], train[:, -n_features]
        test_X, test_y = test[:, :n_obs], test[:, -n_features]
        #print(train_X.shape, len(train_X), train_y.shape)
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], look_back, n_features))
        test_X = test_X.reshape((test_X.shape[0], look_back, n_features))
        #print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
        # design network
        model = Sequential()
        model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
        #model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2]),return_sequences=True))
        #model.add(LSTM(100,return_sequences=True))
        #model.add(LSTM(50,return_sequences=True))
        #model.add(LSTM(100))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer=optimizer_al[opt])
        model.summary()
        # fit network
        history = model.fit(train_X, train_y, epochs=10, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # plot history
        #plt.plot(history.history['loss'], label='train')
        #plt.plot(history.history['val_loss'], label='test')
        #plt.legend()
        #plt.show()
        #figname="Plot"+str(i)+".png"
        #plt.savefig(figname,formatt="png")
        
        # make a prediction of train set
        ytrainpredict = model.predict(train_X)
        train_X = train_X.reshape((train_X.shape[0], look_back*n_features))
        inv_ytrainpredict = concatenate((ytrainpredict, train_X[:, -(n_features-1):]), axis=1)
        inv_ytrainpredict = scaler.inverse_transform(inv_ytrainpredict)
        inv_ytrainpredict = inv_ytrainpredict[:,0]
        train_y = train_y.reshape((len(train_y), 1))
        inv_trainy = concatenate((train_y, train_X[:, -(n_features-1):]), axis=1)
        inv_trainy = scaler.inverse_transform(inv_trainy)
        inv_trainy = inv_trainy[:,0]
        #calculate score
        trainMSE = mean_squared_error(inv_trainy, inv_ytrainpredict)
        
        trainRMSE = math.sqrt(trainMSE)
        
        train_MAPE = mean_absolute_error(inv_trainy, inv_ytrainpredict)/ (sum(inv_trainy)/len(inv_trainy)) * 100
        
        train_SMAPE = smape(inv_trainy, inv_ytrainpredict)
        
        #print('Train MSE: %.3f' %trainMSE)
        #print('Train RMSE: %.3f' %trainRMSE)
        #print('Train MAPE: %.3f' %train_MAPE)
        #print('Train SMAPE: %.3f' %train_SMAPE)
        
        
        # make a prediction of test set
        ytestpredtict = model.predict(test_X)        
        test_X = test_X.reshape((test_X.shape[0], look_back*n_features))
        # invert scaling for forecast
        inv_ytestpredict = concatenate((ytestpredtict, test_X[:, -(n_features-1):]), axis=1)
        inv_ytestpredict = scaler.inverse_transform(inv_ytestpredict)
        inv_ytestpredict = inv_ytestpredict[:,0]
        # invert scaling for actual
        test_y = test_y.reshape((len(test_y), 1))
        inv_y = concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:,0]
        # calculate score
        testMSE = mean_squared_error(inv_y, inv_ytestpredict)
        
        testRMSE = math.sqrt(testMSE)
        
        test_MAPE = mean_absolute_error(inv_y, inv_ytestpredict)/ (sum(inv_y)/len(inv_y)) * 100
        
        test_SMAPE = smape(inv_y, inv_ytestpredict)
        
        #rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
        #print('Test MSE: %.3f' % testMSE)
        #print('Test RMSE: %.3f' % testRMSE)  
        #print('Test MAPE: %.3f' % test_MAPE)
        print('Optimizer: ' + optimizer_al[opt])    
        print('Test SMAPE: %.3f' % test_SMAPE)   
        if (test_SMAPE<min_sMAPE_lstm):
            min_sMAPE_lstm=test_SMAPE
            best_optimizer=opt
print('The best optimizer function: ' + optimizer_al[best_optimizer]+ ", sMAPE: " + str(min_sMAPE_lstm))   

### 3. Turning Epoch

In [ ]:
epoch_ls=[50,60]
#batch_size_ls=[10,20,50,100,200,500,1000]
min_sMAPE_lstm = 100
best_epoch=0

k=0
for ep in range(len(epoch_ls)):    
        #Dataset for LSTM
        dt=np.array([[j] for j in regionWisePickup_porto_data[0]])
        #Take neighbour
        #for h in range(k):
        #    dt1=np.array([[j] for j in regionWisePickup_porto_data[k_neighbour[i][h+1]]])
        #    dt=np.concatenate((dt,dt1),axis=1)
        #Generate a time series with count of POI for stand
        #standpoi=np.full((len(regionWisePickup_porto_data[i]), 1), poi[i])
        #dt=np.concatenate((dt,standpoi),axis=1)
        # ensure all data is float
        values = dt.astype('float32')
        # normalize features
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(values)
        look_back = 5 #number of data look back - to predict, default 1
        # frame as supervised learning
        reframed = series_to_supervised(scaled, look_back, 1)
        # split into train and test sets
        values = reframed.values
        train_size = int(len(values) * 0.7)
        test_size = len(values) - train_size
        #n_train_hours
        train = values[:train_size, :]
        test = values[train_size:, :]
        # split into input and outputs
        n_features = k + 1 # include itself and one extended feature
        #n_features = k + 1
        n_obs = look_back * n_features
        train_X, train_y = train[:, :n_obs], train[:, -n_features]
        test_X, test_y = test[:, :n_obs], test[:, -n_features]
        #print(train_X.shape, len(train_X), train_y.shape)
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], look_back, n_features))
        test_X = test_X.reshape((test_X.shape[0], look_back, n_features))
        #print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
        # design network
        model = Sequential()
        model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
        #model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2]),return_sequences=True))
        #model.add(LSTM(100,return_sequences=True))
        #model.add(LSTM(50,return_sequences=True))
        #model.add(LSTM(100))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='Adamax')
        model.summary()
        # fit network
        history = model.fit(train_X, train_y, epochs=epoch_ls[ep], batch_size=50, validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # plot history
        #plt.plot(history.history['loss'], label='train')
        #plt.plot(history.history['val_loss'], label='test')
        #plt.legend()
        #plt.show()
        #figname="Plot"+str(i)+".png"
        #plt.savefig(figname,formatt="png")
        
        # make a prediction of train set
        ytrainpredict = model.predict(train_X)
        train_X = train_X.reshape((train_X.shape[0], look_back*n_features))
        inv_ytrainpredict = concatenate((ytrainpredict, train_X[:, -(n_features-1):]), axis=1)
        inv_ytrainpredict = scaler.inverse_transform(inv_ytrainpredict)
        inv_ytrainpredict = inv_ytrainpredict[:,0]
        train_y = train_y.reshape((len(train_y), 1))
        inv_trainy = concatenate((train_y, train_X[:, -(n_features-1):]), axis=1)
        inv_trainy = scaler.inverse_transform(inv_trainy)
        inv_trainy = inv_trainy[:,0]
        #calculate score
        trainMSE = mean_squared_error(inv_trainy, inv_ytrainpredict)
        
        trainRMSE = math.sqrt(trainMSE)
        
        train_MAPE = mean_absolute_error(inv_trainy, inv_ytrainpredict)/ (sum(inv_trainy)/len(inv_trainy)) * 100
        
        train_SMAPE = smape(inv_trainy, inv_ytrainpredict)
        
        #print('Train MSE: %.3f' %trainMSE)
        #print('Train RMSE: %.3f' %trainRMSE)
        #print('Train MAPE: %.3f' %train_MAPE)
        #print('Train SMAPE: %.3f' %train_SMAPE)
        
        
        # make a prediction of test set
        ytestpredtict = model.predict(test_X)        
        test_X = test_X.reshape((test_X.shape[0], look_back*n_features))
        # invert scaling for forecast
        inv_ytestpredict = concatenate((ytestpredtict, test_X[:, -(n_features-1):]), axis=1)
        inv_ytestpredict = scaler.inverse_transform(inv_ytestpredict)
        inv_ytestpredict = inv_ytestpredict[:,0]
        # invert scaling for actual
        test_y = test_y.reshape((len(test_y), 1))
        inv_y = concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:,0]
        # calculate score
        testMSE = mean_squared_error(inv_y, inv_ytestpredict)
        
        testRMSE = math.sqrt(testMSE)
        
        test_MAPE = mean_absolute_error(inv_y, inv_ytestpredict)/ (sum(inv_y)/len(inv_y)) * 100
        
        test_SMAPE = smape(inv_y, inv_ytestpredict)
        
        #rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
        #print('Test MSE: %.3f' % testMSE)
        #print('Test RMSE: %.3f' % testRMSE)  
        #print('Test MAPE: %.3f' % test_MAPE)
        print('Epoch: ' + str(epoch_ls[ep]))    
        print('Test SMAPE: %.3f' % test_SMAPE)   
        if (test_SMAPE<min_sMAPE_lstm):
            min_sMAPE_lstm=test_SMAPE
            best_epoch=ep
print('The best epoch: ' + str(epoch_ls[best_epoch])+ ", sMAPE: " + str(min_sMAPE_lstm)) 

In [ ]:
The best epoch: 40, sMAPE: 26.448763
        Epoch: 30
Test SMAPE: 26.667
    Epoch: 25
Test SMAPE: 26.521
    Epoch: 15
Test SMAPE: 26.526
    Epoch: 20
Test SMAPE: 26.540
    Epoch: 10
Test SMAPE: 26.561

### 4. Batch size

In [ ]:
#epoch_ls=[50,60]
batch_size_ls=[10,20,50,100,200,500,1000]
min_sMAPE_lstm = 100
best_batch_size=0

k=0
for bs in range(len(batch_size_ls)):    
        #Dataset for LSTM
        dt=np.array([[j] for j in regionWisePickup_porto_data[0]])
        #Take neighbour
        #for h in range(k):
        #    dt1=np.array([[j] for j in regionWisePickup_porto_data[k_neighbour[i][h+1]]])
        #    dt=np.concatenate((dt,dt1),axis=1)
        #Generate a time series with count of POI for stand
        #standpoi=np.full((len(regionWisePickup_porto_data[i]), 1), poi[i])
        #dt=np.concatenate((dt,standpoi),axis=1)
        # ensure all data is float
        values = dt.astype('float32')
        # normalize features
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(values)
        look_back = 5 #number of data look back - to predict, default 1
        # frame as supervised learning
        reframed = series_to_supervised(scaled, look_back, 1)
        # split into train and test sets
        values = reframed.values
        train_size = int(len(values) * 0.7)
        test_size = len(values) - train_size
        #n_train_hours
        train = values[:train_size, :]
        test = values[train_size:, :]
        # split into input and outputs
        n_features = k + 1 # include itself and one extended feature
        #n_features = k + 1
        n_obs = look_back * n_features
        train_X, train_y = train[:, :n_obs], train[:, -n_features]
        test_X, test_y = test[:, :n_obs], test[:, -n_features]
        #print(train_X.shape, len(train_X), train_y.shape)
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], look_back, n_features))
        test_X = test_X.reshape((test_X.shape[0], look_back, n_features))
        #print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
        # design network
        model = Sequential()
        model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
        #model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2]),return_sequences=True))
        #model.add(LSTM(100,return_sequences=True))
        #model.add(LSTM(50,return_sequences=True))
        #model.add(LSTM(100))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='Adamax')
        model.summary()
        # fit network
        history = model.fit(train_X, train_y, epochs=25, batch_size=batch_size_ls[bs], validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # plot history
        #plt.plot(history.history['loss'], label='train')
        #plt.plot(history.history['val_loss'], label='test')
        #plt.legend()
        #plt.show()
        #figname="Plot"+str(i)+".png"
        #plt.savefig(figname,formatt="png")
        
        # make a prediction of train set
        ytrainpredict = model.predict(train_X)
        train_X = train_X.reshape((train_X.shape[0], look_back*n_features))
        inv_ytrainpredict = concatenate((ytrainpredict, train_X[:, -(n_features-1):]), axis=1)
        inv_ytrainpredict = scaler.inverse_transform(inv_ytrainpredict)
        inv_ytrainpredict = inv_ytrainpredict[:,0]
        train_y = train_y.reshape((len(train_y), 1))
        inv_trainy = concatenate((train_y, train_X[:, -(n_features-1):]), axis=1)
        inv_trainy = scaler.inverse_transform(inv_trainy)
        inv_trainy = inv_trainy[:,0]
        #calculate score
        trainMSE = mean_squared_error(inv_trainy, inv_ytrainpredict)
        
        trainRMSE = math.sqrt(trainMSE)
        
        train_MAPE = mean_absolute_error(inv_trainy, inv_ytrainpredict)/ (sum(inv_trainy)/len(inv_trainy)) * 100
        
        train_SMAPE = smape(inv_trainy, inv_ytrainpredict)
        
        #print('Train MSE: %.3f' %trainMSE)
        #print('Train RMSE: %.3f' %trainRMSE)
        #print('Train MAPE: %.3f' %train_MAPE)
        #print('Train SMAPE: %.3f' %train_SMAPE)
        
        
        # make a prediction of test set
        ytestpredtict = model.predict(test_X)        
        test_X = test_X.reshape((test_X.shape[0], look_back*n_features))
        # invert scaling for forecast
        inv_ytestpredict = concatenate((ytestpredtict, test_X[:, -(n_features-1):]), axis=1)
        inv_ytestpredict = scaler.inverse_transform(inv_ytestpredict)
        inv_ytestpredict = inv_ytestpredict[:,0]
        # invert scaling for actual
        test_y = test_y.reshape((len(test_y), 1))
        inv_y = concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:,0]
        # calculate score
        testMSE = mean_squared_error(inv_y, inv_ytestpredict)
        
        testRMSE = math.sqrt(testMSE)
        
        test_MAPE = mean_absolute_error(inv_y, inv_ytestpredict)/ (sum(inv_y)/len(inv_y)) * 100
        
        test_SMAPE = smape(inv_y, inv_ytestpredict)
        
        #rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
        #print('Test MSE: %.3f' % testMSE)
        #print('Test RMSE: %.3f' % testRMSE)  
        #print('Test MAPE: %.3f' % test_MAPE)
        print('Batch size: ' + str(batch_size_ls[bs]))    
        print('Test SMAPE: %.3f' % test_SMAPE)   
        if (test_SMAPE<min_sMAPE_lstm):
            min_sMAPE_lstm=test_SMAPE
            best_batch_size=bs
print('The best batch size: ' + str(batch_size_ls[best_batch_size])+ ", sMAPE: " + str(min_sMAPE_lstm)) 

### 5. Dropout

In [ ]:
dropout_ls=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
min_sMAPE_lstm = 100
best_dropout=0.0

k=0
for dr in range(len(dropout_ls)):    
        #Dataset for LSTM
        dt=np.array([[j] for j in regionWisePickup_porto_data[0]])
        #Take neighbour
        #for h in range(k):
        #    dt1=np.array([[j] for j in regionWisePickup_porto_data[k_neighbour[i][h+1]]])
        #    dt=np.concatenate((dt,dt1),axis=1)
        #Generate a time series with count of POI for stand
        #standpoi=np.full((len(regionWisePickup_porto_data[i]), 1), poi[i])
        #dt=np.concatenate((dt,standpoi),axis=1)
        # ensure all data is float
        values = dt.astype('float32')
        # normalize features
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(values)
        look_back = 5 #number of data look back - to predict, default 1
        # frame as supervised learning
        reframed = series_to_supervised(scaled, look_back, 1)
        # split into train and test sets
        values = reframed.values
        train_size = int(len(values) * 0.7)
        test_size = len(values) - train_size
        #n_train_hours
        train = values[:train_size, :]
        test = values[train_size:, :]
        # split into input and outputs
        n_features = k + 1 # include itself and one extended feature
        #n_features = k + 1
        n_obs = look_back * n_features
        train_X, train_y = train[:, :n_obs], train[:, -n_features]
        test_X, test_y = test[:, :n_obs], test[:, -n_features]
        #print(train_X.shape, len(train_X), train_y.shape)
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], look_back, n_features))
        test_X = test_X.reshape((test_X.shape[0], look_back, n_features))
        #print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
        # design network
        model = Sequential()
        model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
        #model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2]),return_sequences=True))
        #model.add(LSTM(100,return_sequences=True))
        #model.add(LSTM(50,return_sequences=True))
        #model.add(LSTM(100))
        model.add(Dense(1))
        model.add(Dropout(dropout_ls[dr]))
        model.compile(loss='mae', optimizer='Adamax')
        model.summary()
        # fit network
        history = model.fit(train_X, train_y, epochs=25, batch_size=100, validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # plot history
        #plt.plot(history.history['loss'], label='train')
        #plt.plot(history.history['val_loss'], label='test')
        #plt.legend()
        #plt.show()
        #figname="Plot"+str(i)+".png"
        #plt.savefig(figname,formatt="png")
        
        # make a prediction of train set
        ytrainpredict = model.predict(train_X)
        train_X = train_X.reshape((train_X.shape[0], look_back*n_features))
        inv_ytrainpredict = concatenate((ytrainpredict, train_X[:, -(n_features-1):]), axis=1)
        inv_ytrainpredict = scaler.inverse_transform(inv_ytrainpredict)
        inv_ytrainpredict = inv_ytrainpredict[:,0]
        train_y = train_y.reshape((len(train_y), 1))
        inv_trainy = concatenate((train_y, train_X[:, -(n_features-1):]), axis=1)
        inv_trainy = scaler.inverse_transform(inv_trainy)
        inv_trainy = inv_trainy[:,0]
        #calculate score
        trainMSE = mean_squared_error(inv_trainy, inv_ytrainpredict)
        
        trainRMSE = math.sqrt(trainMSE)
        
        train_MAPE = mean_absolute_error(inv_trainy, inv_ytrainpredict)/ (sum(inv_trainy)/len(inv_trainy)) * 100
        
        train_SMAPE = smape(inv_trainy, inv_ytrainpredict)
        
        #print('Train MSE: %.3f' %trainMSE)
        #print('Train RMSE: %.3f' %trainRMSE)
        #print('Train MAPE: %.3f' %train_MAPE)
        #print('Train SMAPE: %.3f' %train_SMAPE)
        
        
        # make a prediction of test set
        ytestpredtict = model.predict(test_X)        
        test_X = test_X.reshape((test_X.shape[0], look_back*n_features))
        # invert scaling for forecast
        inv_ytestpredict = concatenate((ytestpredtict, test_X[:, -(n_features-1):]), axis=1)
        inv_ytestpredict = scaler.inverse_transform(inv_ytestpredict)
        inv_ytestpredict = inv_ytestpredict[:,0]
        # invert scaling for actual
        test_y = test_y.reshape((len(test_y), 1))
        inv_y = concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:,0]
        # calculate score
        testMSE = mean_squared_error(inv_y, inv_ytestpredict)
        
        testRMSE = math.sqrt(testMSE)
        
        test_MAPE = mean_absolute_error(inv_y, inv_ytestpredict)/ (sum(inv_y)/len(inv_y)) * 100
        
        test_SMAPE = smape(inv_y, inv_ytestpredict)
        
        #rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
        #print('Test MSE: %.3f' % testMSE)
        #print('Test RMSE: %.3f' % testRMSE)  
        #print('Test MAPE: %.3f' % test_MAPE)
        print('Drop out: ' + str(dropout_ls[dr]))    
        print('Test SMAPE: %.3f' % test_SMAPE)   
        if (test_SMAPE<min_sMAPE_lstm):
            min_sMAPE_lstm=test_SMAPE
            best_dropout=dr
print('The best dropout: ' + str(dropout_ls[best_dropout])+ ", sMAPE: " + str(min_sMAPE_lstm)) 

### 6. Number of neuron

In [ ]:
neuron_ls=[10,20,30,40,50,60,70,80,90,100,150,200]
min_sMAPE_lstm = 100
best_neuron=10

k=0
for nr in range(len(neuron_ls)):    
        #Dataset for LSTM
        dt=np.array([[j] for j in regionWisePickup_porto_data[0]])
        #Take neighbour
        #for h in range(k):
        #    dt1=np.array([[j] for j in regionWisePickup_porto_data[k_neighbour[i][h+1]]])
        #    dt=np.concatenate((dt,dt1),axis=1)
        #Generate a time series with count of POI for stand
        #standpoi=np.full((len(regionWisePickup_porto_data[i]), 1), poi[i])
        #dt=np.concatenate((dt,standpoi),axis=1)
        # ensure all data is float
        values = dt.astype('float32')
        # normalize features
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(values)
        look_back = 5 #number of data look back - to predict, default 1
        # frame as supervised learning
        reframed = series_to_supervised(scaled, look_back, 1)
        # split into train and test sets
        values = reframed.values
        train_size = int(len(values) * 0.7)
        test_size = len(values) - train_size
        #n_train_hours
        train = values[:train_size, :]
        test = values[train_size:, :]
        # split into input and outputs
        n_features = k + 1 # include itself and one extended feature
        #n_features = k + 1
        n_obs = look_back * n_features
        train_X, train_y = train[:, :n_obs], train[:, -n_features]
        test_X, test_y = test[:, :n_obs], test[:, -n_features]
        #print(train_X.shape, len(train_X), train_y.shape)
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], look_back, n_features))
        test_X = test_X.reshape((test_X.shape[0], look_back, n_features))
        #print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
        # design network
        model = Sequential()
        model.add(LSTM(neuron_ls[nr], input_shape=(train_X.shape[1], train_X.shape[2])))
        #model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2]),return_sequences=True))
        #model.add(LSTM(100,return_sequences=True))
        #model.add(LSTM(50,return_sequences=True))
        #model.add(LSTM(100))
        model.add(Dense(1))
        model.add(Dropout(0.7))
        model.compile(loss='mae', optimizer='Adamax')
        model.summary()
        # fit network
        history = model.fit(train_X, train_y, epochs=25, batch_size=100, validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # plot history
        #plt.plot(history.history['loss'], label='train')
        #plt.plot(history.history['val_loss'], label='test')
        #plt.legend()
        #plt.show()
        #figname="Plot"+str(i)+".png"
        #plt.savefig(figname,formatt="png")
        
        # make a prediction of train set
        ytrainpredict = model.predict(train_X)
        train_X = train_X.reshape((train_X.shape[0], look_back*n_features))
        inv_ytrainpredict = concatenate((ytrainpredict, train_X[:, -(n_features-1):]), axis=1)
        inv_ytrainpredict = scaler.inverse_transform(inv_ytrainpredict)
        inv_ytrainpredict = inv_ytrainpredict[:,0]
        train_y = train_y.reshape((len(train_y), 1))
        inv_trainy = concatenate((train_y, train_X[:, -(n_features-1):]), axis=1)
        inv_trainy = scaler.inverse_transform(inv_trainy)
        inv_trainy = inv_trainy[:,0]
        #calculate score
        trainMSE = mean_squared_error(inv_trainy, inv_ytrainpredict)
        
        trainRMSE = math.sqrt(trainMSE)
        
        train_MAPE = mean_absolute_error(inv_trainy, inv_ytrainpredict)/ (sum(inv_trainy)/len(inv_trainy)) * 100
        
        train_SMAPE = smape(inv_trainy, inv_ytrainpredict)
        
        #print('Train MSE: %.3f' %trainMSE)
        #print('Train RMSE: %.3f' %trainRMSE)
        #print('Train MAPE: %.3f' %train_MAPE)
        #print('Train SMAPE: %.3f' %train_SMAPE)
        
        
        # make a prediction of test set
        ytestpredtict = model.predict(test_X)        
        test_X = test_X.reshape((test_X.shape[0], look_back*n_features))
        # invert scaling for forecast
        inv_ytestpredict = concatenate((ytestpredtict, test_X[:, -(n_features-1):]), axis=1)
        inv_ytestpredict = scaler.inverse_transform(inv_ytestpredict)
        inv_ytestpredict = inv_ytestpredict[:,0]
        # invert scaling for actual
        test_y = test_y.reshape((len(test_y), 1))
        inv_y = concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:,0]
        # calculate score
        testMSE = mean_squared_error(inv_y, inv_ytestpredict)
        
        testRMSE = math.sqrt(testMSE)
        
        test_MAPE = mean_absolute_error(inv_y, inv_ytestpredict)/ (sum(inv_y)/len(inv_y)) * 100
        
        test_SMAPE = smape(inv_y, inv_ytestpredict)
        
        #rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
        #print('Test MSE: %.3f' % testMSE)
        #print('Test RMSE: %.3f' % testRMSE)  
        #print('Test MAPE: %.3f' % test_MAPE)
        print('Neuron: ' + str(neuron_ls[nr]))    
        print('Test SMAPE: %.3f' % test_SMAPE)   
        if (test_SMAPE<min_sMAPE_lstm):
            min_sMAPE_lstm=test_SMAPE
            best_neuron=nr
print('The best number of neuron: ' + str(neuron_ls[best_neuron])+ ", sMAPE: " + str(min_sMAPE_lstm)) 

### 7. Number of layer 

In [ ]:
n_layer=4
min_sMAPE_lstm = 100
best_layer=1

k=0
for nl in range(0,n_layer):    
        #Dataset for LSTM
        dt=np.array([[j] for j in regionWisePickup_porto_data[0]])
        #Take neighbour
        #for h in range(k):
        #    dt1=np.array([[j] for j in regionWisePickup_porto_data[k_neighbour[i][h+1]]])
        #    dt=np.concatenate((dt,dt1),axis=1)
        #Generate a time series with count of POI for stand
        #standpoi=np.full((len(regionWisePickup_porto_data[i]), 1), poi[i])
        #dt=np.concatenate((dt,standpoi),axis=1)
        # ensure all data is float
        values = dt.astype('float32')
        # normalize features
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(values)
        look_back = 5 #number of data look back - to predict, default 1
        # frame as supervised learning
        reframed = series_to_supervised(scaled, look_back, 1)
        # split into train and test sets
        values = reframed.values
        train_size = int(len(values) * 0.7)
        test_size = len(values) - train_size
        #n_train_hours
        train = values[:train_size, :]
        test = values[train_size:, :]
        # split into input and outputs
        n_features = k + 1 # include itself and one extended feature
        #n_features = k + 1
        n_obs = look_back * n_features
        train_X, train_y = train[:, :n_obs], train[:, -n_features]
        test_X, test_y = test[:, :n_obs], test[:, -n_features]
        #print(train_X.shape, len(train_X), train_y.shape)
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], look_back, n_features))
        test_X = test_X.reshape((test_X.shape[0], look_back, n_features))
        #print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
        # design network
        model = Sequential()
        if (nl==0):
            model.add(LSTM(200, input_shape=(train_X.shape[1], train_X.shape[2])))
            
        if (nl==1):
            model.add(LSTM(200, input_shape=(train_X.shape[1], train_X.shape[2]),return_sequences=True))
            model.add(LSTM(200))
        if (nl==2):
            model.add(LSTM(200, input_shape=(train_X.shape[1], train_X.shape[2]),return_sequences=True))
            model.add(LSTM(200,return_sequences=True))
            model.add(LSTM(200))
        if (nl==3):
            model.add(LSTM(200, input_shape=(train_X.shape[1], train_X.shape[2]),return_sequences=True))
            model.add(LSTM(200,return_sequences=True))
            model.add(LSTM(200,return_sequences=True))
            model.add(LSTM(200))
        model.add(Dense(1))
        model.add(Dropout(0.7))
        model.compile(loss='mae', optimizer='Adamax')
        model.summary()
        # fit network
        history = model.fit(train_X, train_y, epochs=25, batch_size=100, validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # plot history
        #plt.plot(history.history['loss'], label='train')
        #plt.plot(history.history['val_loss'], label='test')
        #plt.legend()
        #plt.show()
        #figname="Plot"+str(i)+".png"
        #plt.savefig(figname,formatt="png")
        
        # make a prediction of train set
        ytrainpredict = model.predict(train_X)
        train_X = train_X.reshape((train_X.shape[0], look_back*n_features))
        inv_ytrainpredict = concatenate((ytrainpredict, train_X[:, -(n_features-1):]), axis=1)
        inv_ytrainpredict = scaler.inverse_transform(inv_ytrainpredict)
        inv_ytrainpredict = inv_ytrainpredict[:,0]
        train_y = train_y.reshape((len(train_y), 1))
        inv_trainy = concatenate((train_y, train_X[:, -(n_features-1):]), axis=1)
        inv_trainy = scaler.inverse_transform(inv_trainy)
        inv_trainy = inv_trainy[:,0]
        #calculate score
        trainMSE = mean_squared_error(inv_trainy, inv_ytrainpredict)
        
        trainRMSE = math.sqrt(trainMSE)
        
        train_MAPE = mean_absolute_error(inv_trainy, inv_ytrainpredict)/ (sum(inv_trainy)/len(inv_trainy)) * 100
        
        train_SMAPE = smape(inv_trainy, inv_ytrainpredict)
        
        #print('Train MSE: %.3f' %trainMSE)
        #print('Train RMSE: %.3f' %trainRMSE)
        #print('Train MAPE: %.3f' %train_MAPE)
        #print('Train SMAPE: %.3f' %train_SMAPE)
        
        
        # make a prediction of test set
        ytestpredtict = model.predict(test_X)        
        test_X = test_X.reshape((test_X.shape[0], look_back*n_features))
        # invert scaling for forecast
        inv_ytestpredict = concatenate((ytestpredtict, test_X[:, -(n_features-1):]), axis=1)
        inv_ytestpredict = scaler.inverse_transform(inv_ytestpredict)
        inv_ytestpredict = inv_ytestpredict[:,0]
        # invert scaling for actual
        test_y = test_y.reshape((len(test_y), 1))
        inv_y = concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:,0]
        # calculate score
        testMSE = mean_squared_error(inv_y, inv_ytestpredict)
        
        testRMSE = math.sqrt(testMSE)
        
        test_MAPE = mean_absolute_error(inv_y, inv_ytestpredict)/ (sum(inv_y)/len(inv_y)) * 100
        
        test_SMAPE = smape(inv_y, inv_ytestpredict)
        
        #rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
        #print('Test MSE: %.3f' % testMSE)
        #print('Test RMSE: %.3f' % testRMSE)  
        #print('Test MAPE: %.3f' % test_MAPE)
        print('Layer: ' + str(nl))    
        print('Test SMAPE: %.3f' % test_SMAPE)   
        if (test_SMAPE<min_sMAPE_lstm):
            min_sMAPE_lstm=test_SMAPE
            best_layer=nl
print('The best number of layer: ' + str(best_layer)+ ", sMAPE: " + str(min_sMAPE_lstm)) 

### 8. Turning k-NN

In [ ]:
kNN=[1,3,5,7,9,11,13,15]
min_sMAPE_lstm = 100
best_k=1
numberOfClusters = 1
#k = 0 #number of neighbours
for k in kNN:    
        #print("Stand: %d" %i)
    #if (i in count_points_new):        
        #print("Number of points: %d" %count_points[i])
        #Dataset for LSTM
        dt=np.array([[j] for j in regionWisePickup_porto_data[0]])
        #Take neighbour
        for h in range(k):
            dt1=np.array([[j] for j in regionWisePickup_porto_data[k_neighbour[i][h+1]]])
            dt=np.concatenate((dt,dt1),axis=1)
        #Generate a time series with count of POI for stand
        #standpoi=np.full((len(regionWisePickup_porto_data[i]), 1), poi[i])
        #dt=np.concatenate((dt,standpoi),axis=1)
        # ensure all data is float
        values = dt.astype('float32')
        # normalize features
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(values)
        look_back = 5 #number of data look back - to predict, default 1
        # frame as supervised learning
        reframed = series_to_supervised(scaled, look_back, 1)
        # split into train and test sets
        values = reframed.values
        train_size = int(len(values) * 0.7)
        test_size = len(values) - train_size
        #n_train_hours
        train = values[:train_size, :]
        test = values[train_size:, :]
        # split into input and outputs
        #n_features = k + 2 # include itself and one extended feature
        n_features = k + 1
        n_obs = look_back * n_features
        train_X, train_y = train[:, :n_obs], train[:, -n_features]
        test_X, test_y = test[:, :n_obs], test[:, -n_features]
        #print(train_X.shape, len(train_X), train_y.shape)
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], look_back, n_features))
        test_X = test_X.reshape((test_X.shape[0], look_back, n_features))
        #print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
        # design network
        model = Sequential()
        model.add(LSTM(200, input_shape=(train_X.shape[1], train_X.shape[2]),return_sequences=True))
        model.add(LSTM(200))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='Adamax')
        model.summary()
        # fit network
        history = model.fit(train_X, train_y, epochs=25, batch_size=100, validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # plot history
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='test')
        plt.legend()
        plt.show()
        figname="Plot_LSTM_Turning_KNN_"+str(k)+".png"
        plt.savefig(figname,formatt="png")
        
        # make a prediction of train set
        ytrainpredict = model.predict(train_X)
        train_X = train_X.reshape((train_X.shape[0], look_back*n_features))
        inv_ytrainpredict = concatenate((ytrainpredict, train_X[:, -(n_features-1):]), axis=1)
        inv_ytrainpredict = scaler.inverse_transform(inv_ytrainpredict)
        inv_ytrainpredict = inv_ytrainpredict[:,0]
        train_y = train_y.reshape((len(train_y), 1))
        inv_trainy = concatenate((train_y, train_X[:, -(n_features-1):]), axis=1)
        inv_trainy = scaler.inverse_transform(inv_trainy)
        inv_trainy = inv_trainy[:,0]
        #calculate score
        trainMSE = mean_squared_error(inv_trainy, inv_ytrainpredict)
        #btrainMSE.append(trainMSE)
        trainRMSE = math.sqrt(trainMSE)
        #btrainRMSE.append(trainRMSE)
        train_MAPE = mean_absolute_error(inv_trainy, inv_ytrainpredict)/ (sum(inv_trainy)/len(inv_trainy)) * 100
        #btrainMAPE.append(train_MAPE)
        train_SMAPE = smape(inv_trainy, inv_ytrainpredict)
        #btrainSMAPE.append(train_SMAPE)        
        #print('Train MSE: %.3f' %trainMSE)
        #print('Train RMSE: %.3f' %trainRMSE)
        #print('Train MAPE: %.3f' %train_MAPE)
        #print('Train SMAPE: %.3f' %train_SMAPE)
        
        
        # make a prediction of test set
        ytestpredtict = model.predict(test_X)        
        test_X = test_X.reshape((test_X.shape[0], look_back*n_features))
        # invert scaling for forecast
        inv_ytestpredict = concatenate((ytestpredtict, test_X[:, -(n_features-1):]), axis=1)
        inv_ytestpredict = scaler.inverse_transform(inv_ytestpredict)
        inv_ytestpredict = inv_ytestpredict[:,0]
        # invert scaling for actual
        test_y = test_y.reshape((len(test_y), 1))
        inv_y = concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:,0]
        # calculate score
        testMSE = mean_squared_error(inv_y, inv_ytestpredict)
        #btestMSE.append(testMSE)
        testRMSE = math.sqrt(testMSE)
        #btestRMSE.append(testRMSE)
        test_MAPE = mean_absolute_error(inv_y, inv_ytestpredict)/ (sum(inv_y)/len(inv_y)) * 100
        #btestMAPE.append(test_MAPE)
        test_SMAPE = smape(inv_y, inv_ytestpredict)
        #btestSMAPE.append(test_SMAPE)    
        #rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
        #print('Test MSE: %.3f' % testMSE)
        #print('Test RMSE: %.3f' % testRMSE)  
        #print('Test MAPE: %.3f' % test_MAPE)
        print('k = ' + str(k))
        print('Test SMAPE: %.3f' % test_SMAPE) 
        if (test_SMAPE<min_sMAPE_lstm):
            min_sMAPE_lstm=test_SMAPE
            best_k=k
print('The best number of kNN: ' + str(best_k)+ ", sMAPE: " + str(min_sMAPE_lstm)) 
        

# LSTM model 

In [ ]:
btrainMSE=[]
btestMSE=[]
btrainRMSE=[]
btestRMSE=[]
btrainMAPE=[]
btestMAPE=[]
btrainSMAPE=[]
btestSMAPE=[]

numberOfClusters = 63
k = 0 #number of neighbours
for i in range(numberOfClusters):    
        print("Stand: %d" %i)
    #if (i in count_points_new):        
        #print("Number of points: %d" %count_points[i])
        #Dataset for LSTM
        #dt=np.array([[j] for j in regionWisePickup_porto_data[i]])
                #Dataset for LSTM
        dt=np.array([[j] for j in regionWisePickup_porto_data[i]])

        #Generate a time series with count of POI for stand
        standpoi=np.array([[j] for j in standvisit[i]])
        
        dt=np.concatenate((dt,standpoi),axis=1)
        #Take neighbour
        #for h in range(k):
        #    dt1=np.array([[j] for j in regionWisePickup_porto_data[k_neighbour[i][h+1]]])
        #    dt=np.concatenate((dt,dt1),axis=1)
        #Generate a time series with count of POI for stand
        #standpoi=np.full((len(regionWisePickup_porto_data[i]), 1), poi[i])
        #dt=np.concatenate((dt,standpoi),axis=1)
        # ensure all data is float
        values = dt.astype('float32')
        # normalize features
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(values)
        look_back = 5 #number of data look back - to predict, default 1
        # frame as supervised learning
        reframed = series_to_supervised(scaled, look_back, 1)
        # split into train and test sets
        values = reframed.values
        train_size = int(len(values) * 0.7)
        test_size = len(values) - train_size
        #n_train_hours
        train = values[:train_size, :]
        test = values[train_size:, :]
        # split into input and outputs
        #n_features = k + 2 # include itself and one extended feature
        n_features = k + 1
        n_obs = look_back * n_features
        train_X, train_y = train[:, :n_obs], train[:, -n_features]
        test_X, test_y = test[:, :n_obs], test[:, -n_features]
        #print(train_X.shape, len(train_X), train_y.shape)
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], look_back, n_features))
        test_X = test_X.reshape((test_X.shape[0], look_back, n_features))
        #print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
        # design network
        model = Sequential()
        model.add(LSTM(200, input_shape=(train_X.shape[1], train_X.shape[2]),return_sequences=True))
        model.add(LSTM(200))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='Adamax')
        model.summary()
        # fit network
        history = model.fit(train_X, train_y, epochs=25, batch_size=100, validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # plot history
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='test')
        plt.legend()
        plt.show()
        figname="Plot_LSTM"+str(i)+".png"
        plt.savefig(figname,formatt="png")
        
        # make a prediction of train set
        ytrainpredict = model.predict(train_X)
        train_X = train_X.reshape((train_X.shape[0], look_back*n_features))
        inv_ytrainpredict = concatenate((ytrainpredict, train_X[:, -(n_features-1):]), axis=1)
        inv_ytrainpredict = scaler.inverse_transform(inv_ytrainpredict)
        inv_ytrainpredict = inv_ytrainpredict[:,0]
        train_y = train_y.reshape((len(train_y), 1))
        inv_trainy = concatenate((train_y, train_X[:, -(n_features-1):]), axis=1)
        inv_trainy = scaler.inverse_transform(inv_trainy)
        inv_trainy = inv_trainy[:,0]
        #calculate score
        trainMSE = mean_squared_error(inv_trainy, inv_ytrainpredict)
        btrainMSE.append(trainMSE)
        trainRMSE = math.sqrt(trainMSE)
        btrainRMSE.append(trainRMSE)
        train_MAPE = mean_absolute_error(inv_trainy, inv_ytrainpredict)/ (sum(inv_trainy)/len(inv_trainy)) * 100
        btrainMAPE.append(train_MAPE)
        train_SMAPE = smape(inv_trainy, inv_ytrainpredict)
        btrainSMAPE.append(train_SMAPE)        
        print('Train MSE: %.3f' %trainMSE)
        print('Train RMSE: %.3f' %trainRMSE)
        print('Train MAPE: %.3f' %train_MAPE)
        print('Train SMAPE: %.3f' %train_SMAPE)
        
        
        # make a prediction of test set
        ytestpredtict = model.predict(test_X)        
        test_X = test_X.reshape((test_X.shape[0], look_back*n_features))
        # invert scaling for forecast
        inv_ytestpredict = concatenate((ytestpredtict, test_X[:, -(n_features-1):]), axis=1)
        inv_ytestpredict = scaler.inverse_transform(inv_ytestpredict)
        inv_ytestpredict = inv_ytestpredict[:,0]
        # invert scaling for actual
        test_y = test_y.reshape((len(test_y), 1))
        inv_y = concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:,0]
        # calculate score
        testMSE = mean_squared_error(inv_y, inv_ytestpredict)
        btestMSE.append(testMSE)
        testRMSE = math.sqrt(testMSE)
        btestRMSE.append(testRMSE)
        test_MAPE = mean_absolute_error(inv_y, inv_ytestpredict)/ (sum(inv_y)/len(inv_y)) * 100
        btestMAPE.append(test_MAPE)
        test_SMAPE = smape(inv_y, inv_ytestpredict)
        btestSMAPE.append(test_SMAPE)    
        #rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
        print('Test MSE: %.3f' % testMSE)
        print('Test RMSE: %.3f' % testRMSE)  
        print('Test MAPE: %.3f' % test_MAPE)
        print('Test SMAPE: %.3f' % test_SMAPE) 
        

In [ ]:
## Print results, 
print("Average train MSE = ", np.mean(btrainMSE))
print("Average test MSE = ", np.mean(btestMSE))
print("Average train MAPE = ", np.mean(btrainMAPE))
print("Average test MAPE = ", np.mean(btestMAPE))
print("Average train SMAPE = ", np.mean(btrainSMAPE))
print("Average test SMAPE = ", np.mean(btestSMAPE))

## LSTM with kNN 

In [ ]:
ctrainMSE=[]
ctestMSE=[]
ctrainRMSE=[]
ctestRMSE=[]
ctrainMAPE=[]
ctestMAPE=[]
ctrainSMAPE=[]
ctestSMAPE=[]

numberOfClusters = 63
k = 15 #number of neighbours
for i in range(numberOfClusters):    
        print("Stand: %d" %i)
    #if (i in count_points_new):        
        #print("Number of points: %d" %count_points[i])
        #Dataset for LSTM
        dt=np.array([[j] for j in regionWisePickup_porto_data[i]])
        #Take neighbour
        for h in range(k):
            dt1=np.array([[j] for j in regionWisePickup_porto_data[k_neighbour[i][h+1]]])
            dt=np.concatenate((dt,dt1),axis=1)
        #Generate a time series with count of POI for stand
        #standpoi=np.full((len(regionWisePickup_porto_data[i]), 1), poi[i])
        #dt=np.concatenate((dt,standpoi),axis=1)
        # ensure all data is float
        values = dt.astype('float32')
        # normalize features
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(values)
        look_back = 5 #number of data look back - to predict, default 1
        # frame as supervised learning
        reframed = series_to_supervised(scaled, look_back, 1)
        # split into train and test sets
        values = reframed.values
        train_size = int(len(values) * 0.7)
        test_size = len(values) - train_size
        #n_train_hours
        train = values[:train_size, :]
        test = values[train_size:, :]
        # split into input and outputs
        #n_features = k + 2 # include itself and one extended feature
        n_features = k + 1
        n_obs = look_back * n_features
        train_X, train_y = train[:, :n_obs], train[:, -n_features]
        test_X, test_y = test[:, :n_obs], test[:, -n_features]
        #print(train_X.shape, len(train_X), train_y.shape)
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], look_back, n_features))
        test_X = test_X.reshape((test_X.shape[0], look_back, n_features))
        #print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
        # design network
        model = Sequential()
        model.add(LSTM(200, input_shape=(train_X.shape[1], train_X.shape[2]),return_sequences=True))
        model.add(LSTM(200))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='Adamax')
        model.summary()
        # fit network
        history = model.fit(train_X, train_y, epochs=25, batch_size=100, validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # plot history
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='test')
        plt.legend()
        plt.show()
        figname="Plot_LSTM_kNN_Test"+str(i)+".png"
        plt.savefig(figname,formatt="png")
        
        # make a prediction of train set
        ytrainpredict = model.predict(train_X)
        train_X = train_X.reshape((train_X.shape[0], look_back*n_features))
        inv_ytrainpredict = concatenate((ytrainpredict, train_X[:, -(n_features-1):]), axis=1)
        inv_ytrainpredict = scaler.inverse_transform(inv_ytrainpredict)
        inv_ytrainpredict = inv_ytrainpredict[:,0]
        train_y = train_y.reshape((len(train_y), 1))
        inv_trainy = concatenate((train_y, train_X[:, -(n_features-1):]), axis=1)
        inv_trainy = scaler.inverse_transform(inv_trainy)
        inv_trainy = inv_trainy[:,0]
        #calculate score
        trainMSE = mean_squared_error(inv_trainy, inv_ytrainpredict)
        ctrainMSE.append(trainMSE)
        trainRMSE = math.sqrt(trainMSE)
        ctrainRMSE.append(trainRMSE)
        train_MAPE = mean_absolute_error(inv_trainy, inv_ytrainpredict)/ (sum(inv_trainy)/len(inv_trainy)) * 100
        ctrainMAPE.append(train_MAPE)
        train_SMAPE = smape(inv_trainy, inv_ytrainpredict)
        ctrainSMAPE.append(train_SMAPE)        
        print('Train MSE: %.3f' %trainMSE)
        print('Train RMSE: %.3f' %trainRMSE)
        print('Train MAPE: %.3f' %train_MAPE)
        print('Train SMAPE: %.3f' %train_SMAPE)
        
        
        # make a prediction of test set
        ytestpredtict = model.predict(test_X)        
        test_X = test_X.reshape((test_X.shape[0], look_back*n_features))
        # invert scaling for forecast
        inv_ytestpredict = concatenate((ytestpredtict, test_X[:, -(n_features-1):]), axis=1)
        inv_ytestpredict = scaler.inverse_transform(inv_ytestpredict)
        inv_ytestpredict = inv_ytestpredict[:,0]
        # invert scaling for actual
        test_y = test_y.reshape((len(test_y), 1))
        inv_y = concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:,0]
        # calculate score
        testMSE = mean_squared_error(inv_y, inv_ytestpredict)
        ctestMSE.append(testMSE)
        testRMSE = math.sqrt(testMSE)
        ctestRMSE.append(testRMSE)
        test_MAPE = mean_absolute_error(inv_y, inv_ytestpredict)/ (sum(inv_y)/len(inv_y)) * 100
        ctestMAPE.append(test_MAPE)
        test_SMAPE = smape(inv_y, inv_ytestpredict)
        ctestSMAPE.append(test_SMAPE)    
        #rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
        print('Test MSE: %.3f' % testMSE)
        print('Test RMSE: %.3f' % testRMSE)  
        print('Test MAPE: %.3f' % test_MAPE)
        print('Test SMAPE: %.3f' % test_SMAPE) 
        

In [ ]:
## Print results, 
print("Average train MSE = ", np.mean(ctrainMSE))
print("Average test MSE = ", np.mean(ctestMSE))
print("Average train MAPE = ", np.mean(ctrainMAPE))
print("Average test MAPE = ", np.mean(ctestMAPE))
print("Average train SMAPE = ", np.mean(ctrainSMAPE))
print("Average test SMAPE = ", np.mean(ctestSMAPE))

## Affect of K-neighbor 

In [ ]:
ctrainMSE=[]
ctestMSE=[]
ctrainRMSE=[]
ctestRMSE=[]
ctrainMAPE=[]
ctestMAPE=[]
ctrainSMAPE=[]
ctestSMAPE=[]

numberOfClusters = 63
#k = 15 #number of neighbours
#Test for stand 11: regionWisePickup_porto_data[10]
k=1
while k<numberOfClusters-1:    
        print("K= %d" %k)
    #if (i in count_points_new):        
        #print("Number of points: %d" %count_points[i])
        #Dataset for LSTM
        dt=np.array([[j] for j in regionWisePickup_porto_data[10]])
        #Take neighbour
        for h in range(k):
            dt1=np.array([[j] for j in regionWisePickup_porto_data[k_neighbour[10][h+1]]])
            dt=np.concatenate((dt,dt1),axis=1)
        #Generate a time series with count of POI for stand
        #standpoi=np.full((len(regionWisePickup_porto_data[i]), 1), poi[i])
        #dt=np.concatenate((dt,standpoi),axis=1)
        # ensure all data is float
        values = dt.astype('float32')
        # normalize features
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(values)
        look_back = 5 #number of data look back - to predict, default 1
        # frame as supervised learning
        reframed = series_to_supervised(scaled, look_back, 1)
        # split into train and test sets
        values = reframed.values
        train_size = int(len(values) * 0.7)
        test_size = len(values) - train_size
        #n_train_hours
        train = values[:train_size, :]
        test = values[train_size:, :]
        # split into input and outputs
        #n_features = k + 2 # include itself and one extended feature
        n_features = k + 1
        n_obs = look_back * n_features
        train_X, train_y = train[:, :n_obs], train[:, -n_features]
        test_X, test_y = test[:, :n_obs], test[:, -n_features]
        #print(train_X.shape, len(train_X), train_y.shape)
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], look_back, n_features))
        test_X = test_X.reshape((test_X.shape[0], look_back, n_features))
        #print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
        # design network
        model = Sequential()
        model.add(LSTM(200, input_shape=(train_X.shape[1], train_X.shape[2]),return_sequences=True))
        model.add(LSTM(200))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='Adamax')
        model.summary()
        # fit network
        history = model.fit(train_X, train_y, epochs=10, batch_size=100, validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # plot history
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='test')
        plt.legend()
        plt.show()
        figname="Plot_LSTM_kNN_Test"+str(i)+".png"
        plt.savefig(figname,formatt="png")
        
        # make a prediction of train set
        ytrainpredict = model.predict(train_X)
        train_X = train_X.reshape((train_X.shape[0], look_back*n_features))
        inv_ytrainpredict = concatenate((ytrainpredict, train_X[:, -(n_features-1):]), axis=1)
        inv_ytrainpredict = scaler.inverse_transform(inv_ytrainpredict)
        inv_ytrainpredict = inv_ytrainpredict[:,0]
        train_y = train_y.reshape((len(train_y), 1))
        inv_trainy = concatenate((train_y, train_X[:, -(n_features-1):]), axis=1)
        inv_trainy = scaler.inverse_transform(inv_trainy)
        inv_trainy = inv_trainy[:,0]
        #calculate score
        trainMSE = mean_squared_error(inv_trainy, inv_ytrainpredict)
        ctrainMSE.append(trainMSE)
        trainRMSE = math.sqrt(trainMSE)
        ctrainRMSE.append(trainRMSE)
        train_MAPE = mean_absolute_error(inv_trainy, inv_ytrainpredict)/ (sum(inv_trainy)/len(inv_trainy)) * 100
        ctrainMAPE.append(train_MAPE)
        train_SMAPE = smape(inv_trainy, inv_ytrainpredict)
        ctrainSMAPE.append(train_SMAPE)        
        print('Train MSE: %.3f' %trainMSE)
        print('Train RMSE: %.3f' %trainRMSE)
        print('Train MAPE: %.3f' %train_MAPE)
        print('Train SMAPE: %.3f' %train_SMAPE)
        
        
        # make a prediction of test set
        ytestpredtict = model.predict(test_X)        
        test_X = test_X.reshape((test_X.shape[0], look_back*n_features))
        # invert scaling for forecast
        inv_ytestpredict = concatenate((ytestpredtict, test_X[:, -(n_features-1):]), axis=1)
        inv_ytestpredict = scaler.inverse_transform(inv_ytestpredict)
        inv_ytestpredict = inv_ytestpredict[:,0]
        # invert scaling for actual
        test_y = test_y.reshape((len(test_y), 1))
        inv_y = concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:,0]
        # calculate score
        testMSE = mean_squared_error(inv_y, inv_ytestpredict)
        ctestMSE.append(testMSE)
        testRMSE = math.sqrt(testMSE)
        ctestRMSE.append(testRMSE)
        test_MAPE = mean_absolute_error(inv_y, inv_ytestpredict)/ (sum(inv_y)/len(inv_y)) * 100
        ctestMAPE.append(test_MAPE)
        test_SMAPE = smape(inv_y, inv_ytestpredict)
        ctestSMAPE.append(test_SMAPE)    
        #rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
        print('Test MSE: %.3f' % testMSE)
        print('Test RMSE: %.3f' % testRMSE)  
        print('Test MAPE: %.3f' % test_MAPE)
        print('Test SMAPE: %.3f' % test_SMAPE)
        
        k=k+2